# 01 - SINGLE-GAME EDA
- This was the single-game notebook I used to examine the columns, clean data, and create new features before applying the same logic to all games.

In [1]:
import numpy as np
import pandas as pd

# Column and row display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)
pd.reset_option('display.max_colwidth')

# Notebook cell width display
from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 98% !important; }</style>"))

# Float appearance, Pandas and NumPy
pd.set_option('display.float_format', '{:.0f}'.format)
np.set_printoptions(suppress=True, precision=0)

# DATA IMPORT
- Let's make a call to the Sportradar API for a single game so that we can get an idea of what the data look like before trying to work with all games in the 2022 season.
- The game example in this case is 2021's week 3 game featuring the Seattle Seahawks at the Minnesota Vikings.

In [2]:
import http.client
import json

conn = http.client.HTTPSConnection("api.sportradar.us")

# 2021, Week 3, SEA at MIN
# 7d06369a-382a-448a-b295-6da9eab53245

# 2022, Week 16, BUF at CIN
# 7c0c0abf-f566-4a1f-a482-b74a991e1663

# 2022, Week 1, BUF at LAR
# bf60c8fc-35cd-4749-a29e-4f48d66da57c

# 2022, Week 1, CLE at CAR
# c4a110f2-847b-4710-9a54-7be66fdb3c99
conn.request("GET", "/nfl/official/trial/v7/en/games/7d06369a-382a-448a-b295-6da9eab53245/pbp.json?api_key={replace_with_your_api_key}")

res = conn.getresponse()
data = res.read()

json_data = json.loads(data.decode("utf-8"))

#### An <u>EXTREMELY</u> complex dataset
- For the sake of my audience, I won't show `json_data` or even a single entry because the data are all in multiple nestings, making the structure very difficult to interpret.
- This will require a lot of exploration to understand.

# DICTIONARY UNNESTING
## Game Data

In [3]:
games = pd.json_normalize(json_data, errors='ignore')

#### Games --> Periods
- The resulting dataframe is one row for the single game we're exploring
- The `periods` column seems to have multiple rows nested within it, so we'll drill into that.
- Having gone through this game's data before, I know there are some fields with the same name across multiple nestings
  - I will rename or delete some of these columns.
- I also know that some of these columns are not relevant to categorizing NFL wide receivers. I will delete these columns. Examples include:
  - Team records
  - Season week
  - Venue information
  - Timeouts used

In [5]:
# Delete unnecessary columns
# 'game_type' used for 2022 season, but not 2021?
games = games.drop(columns = ['status', 'scheduled', 'attendance', 'entry_mode', 'clock', 'quarter', 'conference_game', 'duration', 'periods', '_comment', 'weather.wind.direction', 'summary.season.id', 
       'summary.season.name', 'summary.week.id', 'summary.week.title', 'summary.venue.id', 'summary.venue.name', 'summary.venue.city', 'summary.venue.state', 'summary.venue.country',
       'summary.venue.zip', 'summary.venue.address', 'summary.venue.capacity', 'summary.venue.sr_id', 'summary.venue.location.lat', 'summary.venue.location.lng', 'summary.home.id', 'summary.home.name',
       'summary.home.market', 'summary.home.sr_id', 'summary.home.used_timeouts', 'summary.home.remaining_timeouts', 'summary.home.points', 'summary.home.used_challenges',
       'summary.home.remaining_challenges', 'summary.home.record.wins', 'summary.home.record.losses', 'summary.home.record.ties', 'summary.away.id', 'summary.away.name', 'summary.away.market',
       'summary.away.sr_id', 'summary.away.used_timeouts', 'summary.away.remaining_timeouts', 'summary.away.points', 'summary.away.used_challenges', 'summary.away.remaining_challenges', 'summary.away.record.wins', 'summary.away.record.losses', 'summary.away.record.ties'], errors = 'ignore')

`id` appears in multiple nestings of this dataset, so let's rename it to `game_id` here.

In [6]:
games = games.rename(columns={"id": "game_id"})

In [7]:
games.columns

Index(['game_id', 'sr_id', 'weather.condition', 'weather.humidity',
       'weather.temp', 'weather.wind.speed', 'summary.season.year',
       'summary.season.type', 'summary.week.sequence', 'summary.venue.surface',
       'summary.venue.roof_type', 'summary.home.alias', 'summary.away.alias'],
      dtype='object')

In [8]:
games.head()

,game_id,sr_id,weather.condition,weather.humidity,weather.temp,weather.wind.speed,summary.season.year,summary.season.type,summary.week.sequence,summary.venue.surface,summary.venue.roof_type,summary.home.alias,summary.away.alias
0,7d06369a-382a-448a-b295-6da9eab53245,sr:match:27305854,Controlled Climate,44,77,7,2021,REG,3,artificial,dome,MIN,SEA


## Period Data
- After unnesting `periods`, the data break into 4 rows, each one representing a quarter of a game.
  - If this game had gone to overtime, a new row would be created for the OT period.
  - The `period_type` value would be 'overtime' and the `number` value would be 1.

In [9]:
periods = pd.json_normalize(json_data, record_path=['periods'], errors='ignore')
periods.head()

,period_type,id,number,sequence,pbp,scoring.home.id,scoring.home.name,scoring.home.market,scoring.home.alias,scoring.home.sr_id,scoring.home.points,scoring.away.id,scoring.away.name,scoring.away.market,scoring.away.alias,scoring.away.sr_id,scoring.away.points,coin_toss.home.outcome,coin_toss.home.decision,coin_toss.home.direction,coin_toss.away.outcome,coin_toss.away.decision,coin_toss.away.direction
0,quarter,01ef6fa1-eda6-4689-836e-82dbc78b3e69,1,1,"[{'type': 'drive', 'id': '01ebde76-2065-4995-8...",33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,7,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,10,won,defer,north,lost,receive,north
1,quarter,83b89666-c594-42cb-bff1-47afecda9cb5,2,2,"[{'type': 'drive', 'id': '8928b7d9-711a-41c7-a...",33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,14,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,7,NaN,NaN,NaN,NaN,NaN,NaN
2,quarter,e2f686c4-854f-467e-9db9-1825b04d6a6c,3,3,"[{'type': 'drive', 'id': 'f1e1b3bb-9d19-40c8-9...",33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,3,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,0,won,receive,south,lost,kick,south
3,quarter,2920eec3-0e9c-4dc6-99af-d6ffc21122b5,4,4,"[{'type': 'drive', 'id': '164f459d-6c89-4603-b...",33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,6,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
periods.columns

Index(['period_type', 'id', 'number', 'sequence', 'pbp', 'scoring.home.id',
       'scoring.home.name', 'scoring.home.market', 'scoring.home.alias',
       'scoring.home.sr_id', 'scoring.home.points', 'scoring.away.id',
       'scoring.away.name', 'scoring.away.market', 'scoring.away.alias',
       'scoring.away.sr_id', 'scoring.away.points', 'coin_toss.home.outcome',
       'coin_toss.home.decision', 'coin_toss.home.direction',
       'coin_toss.away.outcome', 'coin_toss.away.decision',
       'coin_toss.away.direction'],
      dtype='object')

#### More columns to drop
- Dropping more columns that are not relevant or have the same names as columns in deeper nestings

In [11]:
periods = periods.drop(columns = ['id', 'sequence', 'scoring.home.name', 'scoring.home.market', 'scoring.home.points', 'scoring.away.name', 'scoring.away.market', 'scoring.away.points', 'coin_toss.home.outcome',
       'coin_toss.home.decision', 'coin_toss.home.direction', 'coin_toss.away.outcome', 'coin_toss.away.decision', 'coin_toss.away.direction'], errors = 'ignore')

In [12]:
periods.head()

,period_type,number,pbp,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id
0,quarter,1,"[{'type': 'drive', 'id': '01ebde76-2065-4995-8...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
1,quarter,2,"[{'type': 'drive', 'id': '8928b7d9-711a-41c7-a...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
2,quarter,3,"[{'type': 'drive', 'id': 'f1e1b3bb-9d19-40c8-9...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
3,quarter,4,"[{'type': 'drive', 'id': '164f459d-6c89-4603-b...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430


## PBP Data

### Access PBP data
- We'll explode the `pbp` column so that each row in the `periods` is multiplied by the number of rows in the `pbp` column for that period

In [13]:
periods_pbp_exploded = periods.explode('pbp')

# Convert pbp column to its own flattened dataframe
pbp = pd.json_normalize(periods_pbp_exploded['pbp'])

In [14]:
periods_pbp_exploded.head()

,period_type,number,pbp,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id
0,quarter,1,"{'type': 'drive', 'id': '01ebde76-2065-4995-88...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
0,quarter,1,"{'type': 'event', 'id': 'ec9240d0-1f08-11ec-a6...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
0,quarter,1,"{'type': 'drive', 'id': 'b67eb032-3253-4577-93...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
0,quarter,1,"{'type': 'event', 'id': '25cf0120-1f0a-11ec-b3...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430
0,quarter,1,"{'type': 'drive', 'id': 'c20efd4b-36b7-4ee9-84...",33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430


In [15]:
pbp.head()

,type,id,sequence,start_reason,end_reason,play_count,duration,first_downs,gain,penalty_yards,inside_20,scoring_drive,created_at,updated_at,team_sequence,start_clock,end_clock,first_drive_yardline,last_drive_yardline,net_yards,pat_successful,pat_points_attempted,events,offensive_team.points,offensive_team.id,defensive_team.points,defensive_team.id,clock,event_type,description,wall_clock,home_points,away_points,play_type,fake_punt,fake_field_goal,screen_pass,play_action,run_pass_option,statistics,details,start_situation.clock,start_situation.down,start_situation.yfd,start_situation.possession.id,start_situation.possession.name,start_situation.possession.market,start_situation.possession.alias,start_situation.possession.sr_id,start_situation.location.id,start_situation.location.name,start_situation.location.market,start_situation.location.alias,start_situation.location.sr_id,start_situation.location.yardline,end_situation.clock,end_situation.down,end_situation.yfd,end_situation.possession.id,end_situation.possession.name,end_situation.possession.market,end_situation.possession.alias,end_situation.possession.sr_id,end_situation.location.id,end_situation.location.name,end_situation.location.market,end_situation.location.alias,end_situation.location.sr_id,end_situation.location.yardline
0,drive,01ebde76-2065-4995-8871-5156d235c3fd,1,Kickoff,Touchdown,9,4:34,4,75,0,True,True,2021-09-26T20:26:01+00:00,2023-07-27T16:06:08+00:00,1,15:00,10:26,75,10,75,True,1,"[{'type': 'play', 'id': 'e62ee710-1eff-11ec-a6...",7,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,0,33405046-04ee-4058-a950-d606f8c30852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,event,ec9240d0-1f08-11ec-a67f-c75e589fa3a4,1632688380473,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-26T20:33:00+00:00,2021-09-26T20:33:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10:26,tv_timeout,TV Timeout,2021-09-26T20:32:58+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,drive,b67eb032-3253-4577-9387-8461866cdec0,2,Kickoff,Touchdown,7,3:49,4,70,0,True,True,2021-09-26T20:35:51+00:00,2023-07-27T16:06:10+00:00,1,10:26,6:37,70,7,70,True,1,"[{'type': 'play', 'id': 'f15ee410-1f08-11ec-a6...",7,33405046-04ee-4058-a950-d606f8c30852,0,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,event,25cf0120-1f0a-11ec-b36d-a3b2acec3de6,1632688906057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-26T20:41:46+00:00,2021-09-26T20:41:46+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6:37,tv_timeout,TV Timeout,2021-09-26T20:41:43+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,drive,c20efd4b-36b7-4ee9-8474-cf681cfee895,3,Kickoff,Field Goal,11,6:10,3,50,-10,NaN,True,2021-09-26T20:44:42+00:00,2023-07-27T16:06:11+00:00,2,6:37,00:27,75,35,40,NaN,0,"[{'type': 'play', 'id': '2a57bd90-1f0a-11ec-b3...",3,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,0,33405046-04ee-4058-a950-d606f8c30852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### PBP data represented by drives
- Each row in the PBP dataset is a drive or possession
  - A drive/possession is a series of plays in which one team possess the ball while the other team is on defense
  - A drive ends when the ball changes possession, a score is made, a game half ends, or the game concludes
- There are also other rows that are not for drives, but for "events," which appear to be TV timeouts and other intermissions

In [16]:
periods_pbp_exploded.columns

Index(['period_type', 'number', 'pbp', 'scoring.home.id', 'scoring.home.alias',
       'scoring.home.sr_id', 'scoring.away.id', 'scoring.away.alias',
       'scoring.away.sr_id'],
      dtype='object')

In [17]:
# Delete unnecessary columns
periods_pbp_exploded = periods_pbp_exploded.drop(columns = ['pbp'])

#### More columns to drop
- Dropping more columns that are not relevant or have the same names as columns in deeper nestings

In [18]:
# Delete unnecessary columns
pbp = pbp.drop(columns = ['home_points', 'away_points', 'play_type', 'fake_punt', 'fake_field_goal', 'screen_pass', 'play_action', 'run_pass_option', 'statistics', 'details', 'start_situation.clock', 'start_situation.down', 'start_situation.yfd', 'start_situation.possession.id', 'start_situation.possession.name',
                          'start_situation.possession.market', 'start_situation.possession.alias', 'start_situation.possession.sr_id', 'start_situation.location.id', 'start_situation.location.name', 'start_situation.location.market', 'start_situation.location.alias', 'start_situation.location.sr_id', 'start_situation.location.yardline',
                          'end_situation.clock', 'end_situation.down', 'end_situation.yfd', 'end_situation.possession.id', 'end_situation.possession.name', 'end_situation.possession.market', 'end_situation.possession.alias', 'end_situation.possession.sr_id', 'end_situation.location.id', 'end_situation.location.name',
                          'end_situation.location.market', 'end_situation.location.alias', 'end_situation.location.sr_id', 'end_situation.location.yardline', 'event_type', 'description', 'type', 'inside_20', 'created_at', 'sequence', 'start_reason', 'end_reason', 'play_count', 'duration', 'first_downs', 'gain', 'penalty_yards', 
                          'scoring_drive', 'created_at', 'updated_at', 'start_clock', 'end_clock', 'first_drive_yardline', 'last_drive_yardline', 'net_yards', 'pat_successful', 'pat_points_attempted', 'offensive_team.points', 'offensive_team.id', 'defensive_team.points', 'defensive_team.id', 'clock', 'wall_clock', 'scoring_play', 
                          'scoring_description', 'hash_mark'], errors = 'ignore')

In [19]:
pbp.head()

,id,team_sequence,events
0,01ebde76-2065-4995-8871-5156d235c3fd,1,"[{'type': 'play', 'id': 'e62ee710-1eff-11ec-a6..."
1,ec9240d0-1f08-11ec-a67f-c75e589fa3a4,NaN,NaN
2,b67eb032-3253-4577-9387-8461866cdec0,1,"[{'type': 'play', 'id': 'f15ee410-1f08-11ec-a6..."
3,25cf0120-1f0a-11ec-b36d-a3b2acec3de6,NaN,NaN
4,c20efd4b-36b7-4ee9-8474-cf681cfee895,2,"[{'type': 'play', 'id': '2a57bd90-1f0a-11ec-b3..."


In [20]:
# Rename id to event_id to avoid confusion
pbp = pbp.rename(columns={"id": "event_id"})

In [21]:
pbp.shape, periods_pbp_exploded.shape

((34, 3), (34, 8))

### Concatenate periods and pbp dataframes
- With the `periods` dataframe expanded to accommodate the exploded `pbp` column we can merge it with the `pbp` dataframe.
- Note the naming convention I'm using so I can keep better track of the steps I've taken thus far and don't get lost along the way:
  - `periods` dataframe, with `pbp` column exploded, merged with `pbp` dataframe
    - `periods_pbp_exploded_pbp`

In [22]:
periods_pbp_exploded = periods_pbp_exploded.reset_index(drop = True)
pbp = pbp.reset_index(drop = True)

periods_pbp_exploded_pbp = pd.concat([periods_pbp_exploded, pbp], axis=1)
periods_pbp_exploded_pbp.head()

,period_type,number,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id,event_id,team_sequence,events
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,"[{'type': 'play', 'id': 'e62ee710-1eff-11ec-a6..."
1,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,ec9240d0-1f08-11ec-a67f-c75e589fa3a4,NaN,NaN
2,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,b67eb032-3253-4577-9387-8461866cdec0,1,"[{'type': 'play', 'id': 'f15ee410-1f08-11ec-a6..."
3,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,25cf0120-1f0a-11ec-b36d-a3b2acec3de6,NaN,NaN
4,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,c20efd4b-36b7-4ee9-8474-cf681cfee895,2,"[{'type': 'play', 'id': '2a57bd90-1f0a-11ec-b3..."


## Events Data
### Access Events data
- As we did before with the `periods` dataframe and `pbp` column, we'll explode the `events` column.
- The naming convention still holds
  - `periods` dataframe, with `pbp` column exploded, merged with `pbp` dataframe, with `events` column exploded
    - `periods_pbp_exploded_pbp_events_exploded`

In [23]:
periods_pbp_exploded_pbp_events_exploded = periods_pbp_exploded_pbp.explode('events')

# Convert events column to its owned flattened dataframe
events_flattened = pd.json_normalize(periods_pbp_exploded_pbp_events_exploded['events'])

In [24]:
periods_pbp_exploded_pbp_events_exploded.shape, events_flattened.shape

((185, 11), (185, 70))

In [25]:
periods_pbp_exploded_pbp_events_exploded.head()

,period_type,number,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id,event_id,team_sequence,events
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,"{'type': 'play', 'id': 'e62ee710-1eff-11ec-a67..."
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,"{'type': 'play', 'id': 'f72f3120-1f07-11ec-a67..."
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,"{'type': 'play', 'id': '1f47ee90-1f08-11ec-a67..."
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,"{'type': 'play', 'id': '27e018c0-1f08-11ec-a67..."
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,"{'type': 'play', 'id': '38edf100-1f08-11ec-a67..."


In [26]:
events_flattened.head(15)

,type,id,sequence,clock,home_points,away_points,play_type,wall_clock,description,fake_punt,fake_field_goal,screen_pass,hash_mark,play_action,run_pass_option,created_at,updated_at,statistics,details,start_situation.clock,start_situation.down,start_situation.yfd,start_situation.possession.id,start_situation.possession.name,start_situation.possession.market,start_situation.possession.alias,start_situation.possession.sr_id,start_situation.location.id,start_situation.location.name,start_situation.location.market,start_situation.location.alias,start_situation.location.sr_id,start_situation.location.yardline,end_situation.clock,end_situation.down,end_situation.yfd,end_situation.possession.id,end_situation.possession.name,end_situation.possession.market,end_situation.possession.alias,end_situation.possession.sr_id,end_situation.location.id,end_situation.location.name,end_situation.location.market,end_situation.location.alias,end_situation.location.sr_id,end_situation.location.yardline,players_rushed,men_in_box,blitz,play_direction,left_tightends,right_tightends,pocket_location,qb_at_snap,huddle,pass_route,running_lane,goaltogo,scoring_play,scoring_description,score.sequence,score.clock,score.points,score.home_points,score.away_points,score.points-after-play.id,score.points-after-play.sequence,score.points-after-play.type,event_type
0,play,e62ee710-1eff-11ec-a67f-c75e589fa3a4,1632687960945,15:00,0,0,kickoff,2021-09-26T20:25:58+00:00,G.Joseph kicks 65 yards from MIN 35 to the SEA...,False,False,False,Middle,False,False,2021-09-26T20:26:00+00:00,2021-09-27T15:00:45+00:00,"[{'stat_type': 'kick', 'attempt': 1, 'yards': ...","[{'category': 'kick_off', 'description': 'G.Jo...",15:00,0,0,33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,35,15:00,1,10,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,play,f72f3120-1f07-11ec-a67f-c75e589fa3a4,1632688011269,15:00,0,0,pass,2021-09-26T20:26:38+00:00,R.Wilson pass short right complete to SEA 36. ...,False,False,False,Middle,True,False,2021-09-26T20:26:51+00:00,2021-09-27T15:03:42+00:00,"[{'stat_type': 'pass', 'attempt': 1, 'complete...","[{'category': 'pass_completion', 'description'...",15:00,1,10,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,25,14:24,1,10,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,42,4,7,False,Right,0,2,Boot Right,Under Center,Huddle,Cross,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,play,1f47ee90-1f08-11ec-a67f-c75e589fa3a4,1632688042628,14:24,0,0,rush,2021-09-26T20:27:16+00:00,C.Carson rushed left tackle to SEA 44 for 2 ya...,False,False,False,Right Hash,False,False,2021-09-26T20:27:22+00:00,2021-09-27T15:04:37+00:00,"[{'stat_type': 'rush', 'attempt': 1, 'yards': ...","[{'category': 'rush', 'description': 'C.Carson...",14:24,1,10,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,42,14:00,2,8,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,44,NaN,6,False,Middle,0,1,NaN,Under Center,Huddle,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,play,27e018c0-1f08-11ec-a67f-c75e589fa3a4,1632688071270,14:00,0,0,rush,2021-09-26T20:27:40+00:00,C.Carson rushed right tackle to MIN 49 for 7 y...,False,False,False,Left Hash,False,True,2021-09-26T20:27:51+00:00,2021-09-27T15:11:07+00:00,"[{'stat_type': 'rush', 'attempt': 1, 'yards': ...","[{'category': 'rush

#### Each event row is a play
- Each event row is a play within a drive. We're getting closer to the data we want to see.

In [27]:
events_flattened.columns

Index(['type', 'id', 'sequence', 'clock', 'home_points', 'away_points',
       'play_type', 'wall_clock', 'description', 'fake_punt',
       'fake_field_goal', 'screen_pass', 'hash_mark', 'play_action',
       'run_pass_option', 'created_at', 'updated_at', 'statistics', 'details',
       'start_situation.clock', 'start_situation.down', 'start_situation.yfd',
       'start_situation.possession.id', 'start_situation.possession.name',
       'start_situation.possession.market', 'start_situation.possession.alias',
       'start_situation.possession.sr_id', 'start_situation.location.id',
       'start_situation.location.name', 'start_situation.location.market',
       'start_situation.location.alias', 'start_situation.location.sr_id',
       'start_situation.location.yardline', 'end_situation.clock',
       'end_situation.down', 'end_situation.yfd',
       'end_situation.possession.id', 'end_situation.possession.name',
       'end_situation.possession.market', 'end_situation.possession.alia

In [28]:
events_flattened = events_flattened.drop(['event_type', 'blitz', 'goaltogo', 'sequence', 'created_at', 'updated_at', 'start_situation.possession.id', 'start_situation.possession.name', 'start_situation.possession.market', 'start_situation.location.id', 'start_situation.location.name', 'start_situation.location.market', 'score.sequence', 'score.clock', 
                                          'score.points', 'score.home_points', 'score.away_points', 'score.points-after-play.id', 'score.points-after-play.sequence', 'score.points-after-play.type', 'event_type', 'running_lane', 'end_situation.location.name',	'end_situation.location.market', 'end_situation.possession.name',
                                          'end_situation.possession.market', 'end_situation.location.id', 'end_situation.location.sr_id', 'start_situation.possession.sr_id', 'start_situation.location.sr_id', 'end_situation.possession.id', 'end_situation.possession.sr_id', 'details'], 
                                         axis = 1, errors = 'ignore')

In [29]:
# Rename events `id` column to `play_id`
events_flattened = events_flattened.rename(columns={"id": "play_id"})

In [30]:
# Drop redundant exploded `events` column from original dataframe
periods_pbp_exploded_pbp_events_exploded = periods_pbp_exploded_pbp_events_exploded.drop('events', axis=1)

### Concatenate periods and events dataframes
- `periods` dataframe, with `pbp` column exploded, merged with `pbp` dataframe, with `events` exploded, merged with `events` dataframe
  - `periods_pbp_exploded_pbp_events_exploded_events`

In [31]:
periods_pbp_exploded_pbp_events_exploded = periods_pbp_exploded_pbp_events_exploded.reset_index(drop = True)
events_flattened = events_flattened.reset_index(drop = True)

periods_pbp_exploded_pbp_events_exploded_events = pd.concat([periods_pbp_exploded_pbp_events_exploded, events_flattened], axis=1)
periods_pbp_exploded_pbp_events_exploded_events.head()

,period_type,number,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id,event_id,team_sequence,type,play_id,clock,home_points,away_points,play_type,wall_clock,description,fake_punt,fake_field_goal,screen_pass,hash_mark,play_action,run_pass_option,statistics,start_situation.clock,start_situation.down,start_situation.yfd,start_situation.possession.alias,start_situation.location.alias,start_situation.location.yardline,end_situation.clock,end_situation.down,end_situation.yfd,end_situation.possession.alias,end_situation.location.alias,end_situation.location.yardline,players_rushed,men_in_box,play_direction,left_tightends,right_tightends,pocket_location,qb_at_snap,huddle,pass_route,scoring_play,scoring_description
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,e62ee710-1eff-11ec-a67f-c75e589fa3a4,15:00,0,0,kickoff,2021-09-26T20:25:58+00:00,G.Joseph kicks 65 yards from MIN 35 to the SEA...,False,False,False,Middle,False,False,"[{'stat_type': 'kick', 'attempt': 1, 'yards': ...",15:00,0,0,MIN,MIN,35,15:00,1,10,SEA,SEA,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,f72f3120-1f07-11ec-a67f-c75e589fa3a4,15:00,0,0,pass,2021-09-26T20:26:38+00:00,R.Wilson pass short right complete to SEA 36. ...,False,False,False,Middle,True,False,"[{'stat_type': 'pass', 'attempt': 1, 'complete...",15:00,1,10,SEA,SEA,25,14:24,1,10,SEA,SEA,42,4,7,Right,0,2,Boot Right,Under Center,Huddle,Cross,NaN,NaN
2,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,1f47ee90-1f08-11ec-a67f-c75e589fa3a4,14:24,0,0,rush,2021-09-26T20:27:16+00:00,C.Carson rushed left tackle to SEA 44 for 2 ya...,False,False,False,Right Hash,False,False,"[{'stat_type': 'rush', 'attempt': 1, 'yards': ...",14:24,1,10,SEA,SEA,42,14:00,2,8,SEA,SEA,44,NaN,6,Middle,0,1,NaN,Under Center,Huddle,NaN,NaN,NaN
3,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,27e018c0-1f08-11ec-a67f-c75e589fa3a4,14:00,0,0,rush,2021-09-26T20:27:40+00:00,C.Carson rushed right tackle to MIN 49 for 7 y...,False,False,False,Left Hash,False,True,"[{'stat_type': 'rush', 'attempt': 1, 'yards': ...",14:00,2,8,SEA,SEA,44,13:26,3,1,SEA,MIN,49,NaN,6,Right,1,0,NaN,Shotgun,No Huddle,NaN,NaN,NaN
4,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,38edf100-1f08-11ec-a67f-c75e589fa3a4,13:26,0,0,rush,2021-09-26T20:28:13+00:00,C.Carson rushed right end to MIN 45 for 4 yard...,False,False,False,Right Hash,False,True,"[{'stat_type': 'rush', 'attempt': 1, 'yards': ...",13:26,3,1,SEA,MIN,49,12:50,1,10,SEA,MIN,45,NaN,7,Right Sideline,0,1,NaN,Shotgun,Huddle,NaN,NaN,NaN


In [32]:
periods_pbp_exploded_pbp_events_exploded_events.shape

(185, 48)

## Statistics Data

In [33]:
periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded = periods_pbp_exploded_pbp_events_exploded_events.explode('statistics')

# Convert events column to its owned flattened dataframe
statistics_flattened = pd.json_normalize(periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded['statistics'])

In [34]:
statistics_flattened.head()

,stat_type,attempt,yards,net_yards,touchback,onside_attempt,onside_success,squib_kick,player.id,player.name,player.jersey,player.position,player.sr_id,team.id,team.name,team.market,team.alias,team.sr_id,category,complete,att_yards,firstdown,inside_20,goaltogo,blitz,hurry,knockdown,pocket_time,on_target_throw,batted_pass,target,reception,yards_after_catch,dropped,catchable,missed_tackles,def_target,def_comp,tackle,broken_tackles,kneel_down,scramble,yards_after_contact,ast_tackle,down,incompletion_type,touchdown,made,penalty,qb_hit,missed,endzone,return,pass_defended,hang_time,faircatch,nullified,sack,sack_yards,ast_sack,tlost,tlost_yards,ast_tlost,fumble,forced,own_rec,own_rec_yards,play_category,forced_fumble,out_of_bounds
0,kick,1,65,40,1,0,0,0,a2aab80d-174c-4639-beac-e2b70bb3625f,Greg Joseph,01,K,sr:player:1184234,33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,return,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,kick_return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,pass,1,17,NaN,NaN,NaN,NaN,NaN,409d4cac-ee90-4470-9710-ebe671678339,Russell Wilson,03,QB,sr:player:831309,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,NaN,1,11,1,0,0,0,1,0,3,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,receive,NaN,17,NaN,NaN,NaN,NaN,NaN,754faf0f-40f7-45f0-b23b-6ce990ecaf26,DK Metcalf,14,WR,sr:player:1230010,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,Seahawks,Seattle,SEA,sr:competitor:4430,NaN,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,6,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,defense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c44b31cd-0480-4aa0-b500-12e9ba0765ac,Nick Vigil,59,LB,sr:player:1074088,33405046-04ee-4058-a950-d606f8c30852,Vikings,Minnesota,MIN,sr:competitor:4423,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,0,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Each row is a player
- Each row represents a player's involvement with the play
  - On a kickoff
    - The kicker
    - The kick returner
  - On a punt
    - The punter
    - The punt returner
  - On a pass play
    - The passer (usually the quarterback, `stat_type` == 'pass')
    - The receiver (`stat_type` == 'receive')
    - The player who tackled the receiver (`stat_type` == 'defense')
  - On a run play
    - The runner (`stat_type` == 'run')
    - The player who tackled the runner (`stat_type` == 'defense')

In [35]:
statistics_flattened.columns

Index(['stat_type', 'attempt', 'yards', 'net_yards', 'touchback',
       'onside_attempt', 'onside_success', 'squib_kick', 'player.id',
       'player.name', 'player.jersey', 'player.position', 'player.sr_id',
       'team.id', 'team.name', 'team.market', 'team.alias', 'team.sr_id',
       'category', 'complete', 'att_yards', 'firstdown', 'inside_20',
       'goaltogo', 'blitz', 'hurry', 'knockdown', 'pocket_time',
       'on_target_throw', 'batted_pass', 'target', 'reception',
       'yards_after_catch', 'dropped', 'catchable', 'missed_tackles',
       'def_target', 'def_comp', 'tackle', 'broken_tackles', 'kneel_down',
       'scramble', 'yards_after_contact', 'ast_tackle', 'down',
       'incompletion_type', 'touchdown', 'made', 'penalty', 'qb_hit', 'missed',
       'endzone', 'return', 'pass_defended', 'hang_time', 'faircatch',
       'nullified', 'sack', 'sack_yards', 'ast_sack', 'tlost', 'tlost_yards',
       'ast_tlost', 'fumble', 'forced', 'own_rec', 'own_rec_yards',
       'pla

In [36]:
statistics_flattened = statistics_flattened.drop(columns = ['missed_tackles', 'def_comp', 'hang_time', 'faircatch', 'nullified', 'sack', 'sack_yards', 'ast_sack', 'tlost', 'tlost_yards', 'ast_tlost', 'fumble', 'forced', 'own_rec', 'own_rec_yards', 'squib_kick', 'onside_attempt', 'onside_success', 'play_category', 
                                                            'forced_fumble', 'out_of_bounds', 'category', 'team.name', 'team.market', 'touchback', 'net_yards', 'kneel_down', 'scramble', 'ast_tackle', 'down', 'made', 'penalty', 'qb_hit', 'missed', 'return', 'pass_defended'], axis = 1, errors = 'ignore')

In [37]:
periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded.shape, statistics_flattened.shape

((576, 48), (576, 34))

In [38]:
# Drop redundant exploded statistics column
periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded = periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded.drop('statistics', axis=1)

### Concatenate pbp and statistics dataframes
- Now we can do one last concatenation to get the statistics data into our working dataframe

In [39]:
periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded = periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded.reset_index(drop = True)
statistics_flattened = statistics_flattened.reset_index(drop = True)

periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded_statistics = pd.concat([periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded, statistics_flattened], axis=1)
periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded_statistics.head()

,period_type,number,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id,event_id,team_sequence,type,play_id,clock,home_points,away_points,play_type,wall_clock,description,fake_punt,fake_field_goal,screen_pass,hash_mark,play_action,run_pass_option,start_situation.clock,start_situation.down,start_situation.yfd,start_situation.possession.alias,start_situation.location.alias,start_situation.location.yardline,end_situation.clock,end_situation.down,end_situation.yfd,end_situation.possession.alias,end_situation.location.alias,end_situation.location.yardline,players_rushed,men_in_box,play_direction,left_tightends,right_tightends,pocket_location,qb_at_snap,huddle,pass_route,scoring_play,scoring_description,stat_type,attempt,yards,player.id,player.name,player.jersey,player.position,player.sr_id,team.id,team.alias,team.sr_id,complete,att_yards,firstdown,inside_20,goaltogo,blitz,hurry,knockdown,pocket_time,on_target_throw,batted_pass,target,reception,yards_after_catch,dropped,catchable,def_target,tackle,broken_tackles,yards_after_contact,incompletion_type,touchdown,endzone
0,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,e62ee710-1eff-11ec-a67f-c75e589fa3a4,15:00,0,0,kickoff,2021-09-26T20:25:58+00:00,G.Joseph kicks 65 yards from MIN 35 to the SEA...,False,False,False,Middle,False,False,15:00,0,0,MIN,MIN,35,15:00,1,10,SEA,SEA,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kick,1,65,a2aab80d-174c-4639-beac-e2b70bb3625f,Greg Joseph,01,K,sr:player:1184234,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,e62ee710-1eff-11ec-a67f-c75e589fa3a4,15:00,0,0,kickoff,2021-09-26T20:25:58+00:00,G.Joseph kicks 65 yards from MIN 35 to the SEA...,False,False,False,Middle,False,False,15:00,0,0,MIN,MIN,35,15:00,1,10,SEA,SEA,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,f72f3120-1f07-11ec-a67f-c75e589fa3a4,15:00,0,0,pass,2021-09-26T20:26:38+00:00,R.Wilson pass short right complete to SEA 36. ...,False,False,False,Middle,True,False,15:00,1,10,SEA,SEA,25,14:24,1,10,SEA,SEA,42,4,7,Right,0,2,Boot Right,Under Center,Huddle,Cross,NaN,NaN,pass,1,17,409d4cac-ee90-4470-9710-ebe671678339,Russell Wilson,03,QB,sr:player:831309,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,1,11,1,0,0,0,1,0,3,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,f72f3120-1f07-11ec-a67f-c75e589fa3a4,15:00,0,0,pass,2021-09-26T20:26:38+00:00,R.Wilson pass short right complete to SEA 36. ...,False,False,False,Middle,True,False,15:00,1,10,SEA,SEA,25,14:24,1,10,SEA,SEA,42,4,7,Right,0,2,Boot Right,Under Center,Huddle,Cross,NaN,NaN,receive,NaN,17,754faf0f-40f7-45f0-b23b-6ce990ecaf26,DK Metcalf,14,WR,sr:player:1230010,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,NaN,NaN,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,1,1,6,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,f72f3120-1f07-11ec-a67f-c75e589fa3a4,15:00,0,0,pass,2021-09-26T20:26:38+00:00,R.Wi

# MERGE DATAFRAMES
- Now we can merge this concatenated dataframe to the `games` dataframe we created earlier.
- The `games` dataframe will have rows below it with missing data, so we'll have to do a forward-fill to populate those.

In [40]:
games = games.reset_index(drop = True)
periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded_statistics = periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded_statistics.reset_index(drop = True)

sportradar = pd.concat([games, periods_pbp_exploded_pbp_events_exploded_events_statistics_exploded_statistics], axis=1)

In [42]:
# Max column width so we can read play descriptions
pd.set_option('display.max_colwidth', None)

In [43]:
# Check for duplicate columns
column_names = list(sportradar.columns)
duplicate_columns = set([x for x in column_names if column_names.count(x) > 1])
duplicate_columns_list = list(duplicate_columns)
duplicate_columns_list

[]

Forward-filling here to make sure every row has the game data from top row

In [44]:
# Forward-fill rows with null values for game columns

# Columns to forward fill
columns_to_fill = [
    'game_id', 'weather.condition', 'weather.humidity', 'weather.temp', 'weather.wind.speed', 'summary.season.year', 'summary.season.type',
    'summary.week.sequence', 'summary.venue.surface', 'summary.venue.roof_type', 'summary.home.alias', 'summary.away.alias'
]

# Applying forward fill to the specified columns
sportradar[columns_to_fill] = sportradar[columns_to_fill].ffill()

# Displaying the first few rows to verify the changes
sportradar.head()

,game_id,sr_id,weather.condition,weather.humidity,weather.temp,weather.wind.speed,summary.season.year,summary.season.type,summary.week.sequence,summary.venue.surface,summary.venue.roof_type,summary.home.alias,summary.away.alias,period_type,number,scoring.home.id,scoring.home.alias,scoring.home.sr_id,scoring.away.id,scoring.away.alias,scoring.away.sr_id,event_id,team_sequence,type,play_id,clock,home_points,away_points,play_type,wall_clock,description,fake_punt,fake_field_goal,screen_pass,hash_mark,play_action,run_pass_option,start_situation.clock,start_situation.down,start_situation.yfd,start_situation.possession.alias,start_situation.location.alias,start_situation.location.yardline,end_situation.clock,end_situation.down,end_situation.yfd,end_situation.possession.alias,end_situation.location.alias,end_situation.location.yardline,players_rushed,men_in_box,play_direction,left_tightends,right_tightends,pocket_location,qb_at_snap,huddle,pass_route,scoring_play,scoring_description,stat_type,attempt,yards,player.id,player.name,player.jersey,player.position,player.sr_id,team.id,team.alias,team.sr_id,complete,att_yards,firstdown,inside_20,goaltogo,blitz,hurry,knockdown,pocket_time,on_target_throw,batted_pass,target,reception,yards_after_catch,dropped,catchable,def_target,tackle,broken_tackles,yards_after_contact,incompletion_type,touchdown,endzone
0,7d06369a-382a-448a-b295-6da9eab53245,sr:match:27305854,Controlled Climate,44,77,7,2021,REG,3,artificial,dome,MIN,SEA,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,e62ee710-1eff-11ec-a67f-c75e589fa3a4,15:00,0,0,kickoff,2021-09-26T20:25:58+00:00,G.Joseph kicks 65 yards from MIN 35 to the SEA End Zone. Touchback.,False,False,False,Middle,False,False,15:00,0,0,MIN,MIN,35,15:00,1,10,SEA,SEA,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,kick,1,65,a2aab80d-174c-4639-beac-e2b70bb3625f,Greg Joseph,01,K,sr:player:1184234,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d06369a-382a-448a-b295-6da9eab53245,NaN,Controlled Climate,44,77,7,2021,REG,3,artificial,dome,MIN,SEA,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,e62ee710-1eff-11ec-a67f-c75e589fa3a4,15:00,0,0,kickoff,2021-09-26T20:25:58+00:00,G.Joseph kicks 65 yards from MIN 35 to the SEA End Zone. Touchback.,False,False,False,Middle,False,False,15:00,0,0,MIN,MIN,35,15:00,1,10,SEA,SEA,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,return,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7d06369a-382a-448a-b295-6da9eab53245,NaN,Controlled Climate,44,77,7,2021,REG,3,artificial,dome,MIN,SEA,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-5156d235c3fd,1,play,f72f3120-1f07-11ec-a67f-c75e589fa3a4,15:00,0,0,pass,2021-09-26T20:26:38+00:00,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.,False,False,False,Middle,True,False,15:00,1,10,SEA,SEA,25,14:24,1,10,SEA,SEA,42,4,7,Right,0,2,Boot Right,Under Center,Huddle,Cross,NaN,NaN,pass,1,17,409d4cac-ee90-4470-9710-ebe671678339,Russell Wilson,03,QB,sr:player:831309,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,1,11,1,0,0,0,1,0,3,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7d06369a-382a-448a-b295-6da9eab53245,NaN,Controlled Climate,44,77,7,2021,REG,3,artificial,dome,MIN,SEA,quarter,1,33405046-04ee-4058-a950-d606f8c30852,MIN,sr:competitor:4423,3d08af9e-c767-4f88-a7dc-b920c6d2b4a8,SEA,sr:competitor:4430,01ebde76-2065-4995-8871-

## Modify dataset
- We're now filtering our dataframe for pass plays
  - We want the rows for the passer, receiver, and defensive pass interferences
     - DPIs are a good indication of a receiver's skills because a defender feels he must commit DPI to defend against a catch

In [45]:
# Filter for padd and receive plays
sportradar = sportradar[
                        (sportradar['stat_type'] == 'pass') 
                        | (sportradar['stat_type'] == 'receive') 
                        | (
                           (sportradar['stat_type'] == 'penalty')
                           & (sportradar['description'].str.contains('defensive pass interference', case = False, na = False))
                          )
                        ]

### Transfer passer-specific columns to receiver-specific rows
- Here, we create new dummy columns for the passers and defenders if they committed DPI

In [46]:
# Create dummy column for quarterback name
sportradar['alias'] = sportradar.loc[sportradar['stat_type'] == 'pass']['player.name']
sportradar['alias'] = sportradar['alias'].apply(str)

# Create dummy column for defender name
sportradar['name'] = sportradar.loc[sportradar['stat_type'] == 'penalty']['player.name']
sportradar['name'] = sportradar['name'].apply(str)

In [47]:
sportradar['alias'].unique(), sportradar['alias'].dtype

(array(['Russell Wilson', 'nan', 'Kirk Cousins'], dtype=object), dtype('O'))

- We checked to make sure that Russell Wilson and Kirk Cousins are the only unique values for this column.
- They were the only passers in this game.

In [48]:
# # Loop through the DataFrame and transfer passer data to receiver rows
# for idx, row in sportradar.iterrows():
#     if row['stat_type'] == 'pass':
#         # Find the corresponding receiver row for the same play
#         receiver_idx = sportradar[(sportradar['play_id'] == row['play_id']) & (sportradar['stat_type'] == 'receive')].index
#         # Update the receiver row with the passer data
#         if len(receiver_idx) > 0:
#             sportradar.loc[receiver_idx, 'qb_complete'] = row['complete']
#             sportradar.loc[receiver_idx, 'qb_att_yards'] = row['att_yards']
#             sportradar.loc[receiver_idx, 'qb_blitz'] = row['c']
#             sportradar.loc[receiver_idx, 'qb_hurry'] = row['hurry']
#             sportradar.loc[receiver_idx, 'qb_knockdown'] = row['knockdown']
#             sportradar.loc[receiver_idx, 'qb_pocket_time'] = row['pocket_time']
#             sportradar.loc[receiver_idx, 'qb_on_target_throw'] = row['on_target_throw']
#             sportradar.loc[receiver_idx, 'qb_batted_pass'] = row['batted_pass']
#             sportradar.loc[receiver_idx, 'qb_incompletion_type'] = row['incompletion_type']
#             sportradar.loc[receiver_idx, 'qb_name'] = row['alias']
            
# for idx, row in sportradar.iterrows():
#     if row['stat_type'] == 'penalty':
#         # Find the corresponding receiver row for the same play
#         defender_idx = sportradar[(sportradar['play_id'] == row['play_id']) & (sportradar['stat_type'] == 'receive')].index
#         # Update the receiver row with the passer data
#         if len(receiver_idx) > 0:
#             sportradar.loc[defender_idx, 'defender_name'] = row['name']

# # Create separate dataframe
# sportradar_wr = sportradar[sportradar['stat_type'] == 'receive'].copy()

# # Drop passer-specific columns in new dataframe
# sportradar_wr.drop(['complete', 'att_yards', 'blitz', 'hurry', 'knockdown', 'pocket_time', 'on_target_throw', 'batted_pass', 'incompletion_type'], axis=1, inplace=True, errors = 'ignore')

#### Passer data
- Here, we create a separate passer data dataframe
- We also create new columns that are just renamed versions of passer-specific columns
  - Although these fields relate to the passer, they could still be helpful in understanding how a receiver plays
    - e.g. Can a receiver be a primary target to "bail out" the QB when he's under duress (blitzed, hurried, or knocked down)?

In [49]:
# Extract passer data
passer_data = sportradar[sportradar['stat_type'] == 'pass'].copy()

# Rename passer columns
passer_columns = {
    'complete': 'qb_complete', 
    'att_yards': 'qb_att_yards', 
    'blitz': 'qb_blitz',
    'hurry': 'qb_hurry',
    'knockdown': 'qb_knockdown',
    'pocket_time': 'qb_pocket_time',
    'on_target_throw': 'qb_on_target_throw',
    'batted_pass': 'qb_batted_pass',
    'incompletion_type': 'qb_incompletion_type',
    'alias': 'qb_name'
}
passer_data.rename(columns=passer_columns, inplace=True)

#### Defender data
- We do the same for defenders too

In [50]:
# Extract defender data
defender_data = sportradar[sportradar['stat_type'] == 'penalty'].copy()

# Rename defender columns (assuming 'name' represents defender's name)
defender_data.rename(columns={'name': 'defender_name'}, inplace=True)

#### Bringing the passer data into the receiver dataframe

In [51]:
# Extract receiver data and create a separate dataframe for it
sportradar_wr = sportradar[sportradar['stat_type'] == 'receive'].copy()

# Merge receiver data with passer data
# Can merge on the `play_id` field
sportradar_wr = sportradar_wr.merge(passer_data[['play_id', 'qb_complete', 'qb_att_yards', 'qb_blitz', 'qb_hurry', 'qb_knockdown', 'qb_pocket_time', 'qb_on_target_throw', 'qb_batted_pass', 'qb_incompletion_type', 'qb_name'
                                                ]], on='play_id', how='left')

# Merge receiver data with defender data
sportradar_wr = sportradar_wr.merge(defender_data[['play_id', 'defender_name']], on='play_id', how='left')

# Drop old passer-specific columns
sportradar_wr.drop(['complete', 'att_yards', 'blitz', 'hurry', 'knockdown', 'pocket_time', 'on_target_throw', 'batted_pass', 'incompletion_type', 'alias', 'name'], axis=1, inplace=True, errors = 'ignore')

We effectively filtered the game dataframe for the `receive` plays and then pulled in select passer-specific columns that are relevant to a receiver's playing style.

In [52]:
sportradar_wr.columns

Index(['game_id', 'sr_id', 'weather.condition', 'weather.humidity',
       'weather.temp', 'weather.wind.speed', 'summary.season.year',
       'summary.season.type', 'summary.week.sequence', 'summary.venue.surface',
       'summary.venue.roof_type', 'summary.home.alias', 'summary.away.alias',
       'period_type', 'number', 'scoring.home.id', 'scoring.home.alias',
       'scoring.home.sr_id', 'scoring.away.id', 'scoring.away.alias',
       'scoring.away.sr_id', 'event_id', 'team_sequence', 'type', 'play_id',
       'clock', 'home_points', 'away_points', 'play_type', 'wall_clock',
       'description', 'fake_punt', 'fake_field_goal', 'screen_pass',
       'hash_mark', 'play_action', 'run_pass_option', 'start_situation.clock',
       'start_situation.down', 'start_situation.yfd',
       'start_situation.possession.alias', 'start_situation.location.alias',
       'start_situation.location.yardline', 'end_situation.clock',
       'end_situation.down', 'end_situation.yfd',
       'end_situa

On DPI calls, the reception will be recorded as a null instead of a 0. Let's change these nulls to zeroes.

In [53]:
# Change reception nulls to zeros
sportradar_wr['reception'].fillna(0, inplace = True)

### Delete columns that aren't in all games
- Some game datasets have columns that differ slightly

In [54]:
sportradar_wr = sportradar_wr.drop(columns = ['game_type','weather.condition', 'lost','opp_rec','opp_rec_yards','interception','int_touchdown','int_yards'], errors = 'ignore')

### Delete unnecesary columns
- There are other versions of these columns that effectively say the same thing. These can be dropped.
- Examples include home team, away team, player ID, game clock, and description
- I know these columns are redundant having looked at all the columns before.

In [55]:
sportradar_wr = sportradar_wr.drop(columns = ['sr_id', 'type', 'play_type', 'stat_type', 'scoring.home.id', 'scoring.away.id', 'player.id', 'team.id', 'scoring.home.alias', 'scoring.away.alias', 'clock', 'wall_clock', 'scoring_description', 'attempt', 'start_situation_possession_alias',
                                                'end_situation_possession_alias',], errors = 'ignore')

### Suffix removal
- Removing some suffixes that only take up more column width
- Also replacing periods with dashes

In [56]:
sportradar_wr = sportradar_wr.rename(columns = {col: col.replace('weather.', '') for col in sportradar_wr.columns})
# sportradar_wr.columns

In [57]:
sportradar_wr = sportradar_wr.rename(columns = {col: col.replace('summary.', '') for col in sportradar_wr.columns})
# sportradar_wr.columns

In [58]:
sportradar_wr = sportradar_wr.rename(columns = {col: col.replace('scoring.', '') for col in sportradar_wr.columns})
# sportradar_wr.columns

In [59]:
sportradar_wr = sportradar_wr.rename(columns = {col: col.replace('.', '_') for col in sportradar_wr.columns})
# sportradar_wr.columns

### Overtime adjustment
- For simplicity's sake, we should change the overtime period number from 1 to 5, and then drop the `period_type` column altogether.

In [60]:
sportradar_wr.loc[(sportradar_wr['period_type'] == 'overtime') & (sportradar_wr['number'] == 1), 'number'] = 5
sportradar_wr = sportradar_wr.drop(columns = ['period_type'], errors = 'ignore')

Tidying up some field names

In [61]:
sportradar_wr = sportradar_wr.rename(columns={'week_sequence': 'week', 'number': 'period', 'start_situation_clock': 'start_play_clock', 'start_situation_down': 'start_play_down', 'start_situation_yfd': 'start_play_yfd', 'start_situation_possession_alias': 'start_play_possession_alias', 'start_situation_location_yardline': 'start_play_yardline', 
                                              'start_situation_location_alias': 'start_play_field_side', 'end_situation_clock': 'end_play_clock', 'end_situation_down': 'end_play_down', 'end_situation_yfd': 'end_play_yfd', 'end_situation_possession_alias': 'end_play_possession_alias', 'end_situation_location_yardline': 'end_play_yardline', 
                                              'end_situation_location_alias': 'end_play_field_side', 'qb_blitz': 'blitz', 'qb_hurry': 'hurry', 'qb_knockdown': 'knockdown', 'qb_pocket_time': 'pocket_time', 'qb_complete': 'complete', 'qb_on_target_throw': 'on_target_throw', 'qb_batted_pass': 'batted_pass', 'qb_incompletion_type': 'incompletion_type', 'qb_att_yards': 'att_yards'})

In [62]:
sportradar_wr.columns

Index(['game_id', 'humidity', 'temp', 'wind_speed', 'season_year',
       'season_type', 'week', 'venue_surface', 'venue_roof_type', 'home_alias',
       'away_alias', 'period', 'home_sr_id', 'away_sr_id', 'event_id',
       'team_sequence', 'play_id', 'home_points', 'away_points', 'description',
       'fake_punt', 'fake_field_goal', 'screen_pass', 'hash_mark',
       'play_action', 'run_pass_option', 'start_play_clock', 'start_play_down',
       'start_play_yfd', 'start_play_possession_alias',
       'start_play_field_side', 'start_play_yardline', 'end_play_clock',
       'end_play_down', 'end_play_yfd', 'end_play_possession_alias',
       'end_play_field_side', 'end_play_yardline', 'players_rushed',
       'men_in_box', 'play_direction', 'left_tightends', 'right_tightends',
       'pocket_location', 'qb_at_snap', 'huddle', 'pass_route', 'scoring_play',
       'yards', 'player_name', 'player_jersey', 'player_position',
       'player_sr_id', 'team_alias', 'team_sr_id', 'firstdown', '

### Clock columns
- Creating string versions of the play clock start, play clock end, and pocket times for easier readability
- Converting the original string versions of the play clock fields to timedeltas.

In [63]:
# For time display in string format
sportradar_wr['start_play_clock_string'] = sportradar_wr['start_play_clock']
sportradar_wr['end_play_clock_string'] = sportradar_wr['end_play_clock']
sportradar_wr['pocket_time_string'] = sportradar_wr['pocket_time'].map('{:,.2f}'.format)

# To convert to timedelta
sportradar_wr['start_play_clock'] = pd.to_timedelta('00:' + sportradar_wr['start_play_clock'])
sportradar_wr['end_play_clock'] = pd.to_timedelta('00:' + sportradar_wr['end_play_clock'])

In [64]:
sportradar_wr['start_play_clock_string'].head()

0    15:00
1    12:50
2    11:24
3    10:32
4     9:51
Name: start_play_clock_string, dtype: object

### Yardline adjustment
- If the possession team is on its side of the field, 50 yards is added to the yardline
  - This will make it easier to verify play distances

In [65]:
sportradar_wr.loc[(sportradar_wr['team_alias']) == (sportradar_wr['start_play_field_side'])][['start_play_field_side', 'team_alias', 'start_play_clock', 'start_play_down', 'start_play_yardline', 'end_play_yardline', 'description']]

,start_play_field_side,team_alias,start_play_clock,start_play_down,start_play_yardline,end_play_yardline,description
0,SEA,SEA,0 days 00:15:00,1,25,42,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.
4,MIN,MIN,0 days 00:09:51,2,37,43,K.Cousins pass short left complete to MIN 35. Catch made by A.Mattison at MIN 35. Gain of 20 yards. Tackled by J.Adams at SEA 43.
9,SEA,SEA,0 days 00:05:14,3,28,38,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.
10,SEA,SEA,0 days 00:04:38,1,38,50,R.Wilson pass short middle complete to SEA 50. Catch made by D.Metcalf at SEA 50. Gain of 12 yards. Tackled by B.Breeland at MIN 50.
13,MIN,MIN,0 days 00:15:00,2,27,27,K.Cousins steps back to pass. K.Cousins pass incomplete short right intended for T.Conklin.
14,MIN,MIN,0 days 00:14:56,3,27,27,K.Cousins steps back to pass. K.Cousins pass incomplete short right intended for A.Thielen (C.Dunlap).
15,SEA,SEA,0 days 00:14:09,1,35,44,R.Wilson pass deep left complete to MIN 45. Catch made by D.Metcalf at MIN 45. Gain of 21 yards. Pushed out of bounds by X.Woods at MIN 44.
16,MIN,MIN,0 days 00:11:07,1,15,21,K.Cousins pass short middle complete to MIN 18. Catch made by A.Mattison at MIN 18. Gain of 6 yards. Tackled by J.Brooks at MIN 21.
17,MIN,MIN,0 days 00:09:40,1,27,50,K.Cousins pass short right complete to MIN 24. Catch made by A.Mattison at MIN 24. Gain of 23 yards. Tackled by U.Amadi at MIN 50.
22,SEA,SEA,0 days 00:05:23,1,21,28,R.Wilson pass short middle complete to SEA 17. Catch made by P.Hart at SEA 17. Gain of 7 yards. Tackled by H.Smith at SEA 28.


- If the Seahawks have the ball and the ball is on the Seahawks 32 yard line, the effective yard line is 68.
  - 50 yards of opponent territory + 18 yards away from midfield line = 68 effective yard line

In [66]:
sportradar_wr['effective_start_play_yardline'] = sportradar_wr['start_play_yardline']
sportradar_wr.loc[(sportradar_wr['team_alias']) == (sportradar_wr['start_play_field_side']), 'effective_start_play_yardline'] = 50 - sportradar_wr['start_play_yardline'] + 50

- If the Seahawks have the ball and then end the play on the the Seahawks 39 yard line, the effective yard line is 61.
  - 50 yards of opponent territory + 11 yards away from midfield line = 61 effective yard line

In [67]:
sportradar_wr['effective_end_play_yardline'] = sportradar_wr['end_play_yardline']
sportradar_wr.loc[(sportradar_wr['team_alias']) == (sportradar_wr['end_play_field_side']), 'effective_end_play_yardline'] = 50 - sportradar_wr['end_play_yardline'] + 50

### Re-ordering columns

In [68]:
sportradar_wr = sportradar_wr[['game_id','season_year','season_type','week','home_alias' ,'home_sr_id','away_alias' ,'away_sr_id', 'temp' ,'humidity' ,'wind_speed' ,'venue_surface' ,'venue_roof_type' ,'period' ,'event_id',
                                'team_sequence','play_id','home_points' ,'away_points' ,'description','start_play_clock', 'start_play_clock_string', 'start_play_down' ,'start_play_yfd' ,'start_play_yardline', 'effective_start_play_yardline', 'start_play_field_side', 'inside_20' ,
                                'goaltogo' ,'end_play_clock','end_play_clock_string', 'end_play_down','end_play_yfd', 'end_play_yardline', 'effective_end_play_yardline', 'end_play_field_side', 'firstdown','scoring_play','players_rushed','men_in_box','huddle',
                                'hash_mark' ,'qb_at_snap' ,'left_tightends' ,'right_tightends', 'qb_name', 'pocket_location' ,'play_direction', 'screen_pass' ,'play_action' ,'run_pass_option' ,'pass_route' ,'fake_punt' ,'fake_field_goal',
                                'defender_name', 'player_name' ,'player_jersey','player_position','player_sr_id','team_alias' ,'team_sr_id', 'blitz' ,'hurry' ,'knockdown' ,'pocket_time', 'pocket_time_string', 'on_target_throw' ,'batted_pass',
                                'incompletion_type' ,'target' ,'reception' ,'yards' ,'att_yards' ,'yards_after_catch', 'yards_after_contact' ,'broken_tackles' ,'dropped','catchable', 'touchdown']]

In [69]:
sportradar_wr.head()

,game_id,season_year,season_type,week,home_alias,home_sr_id,away_alias,away_sr_id,temp,humidity,wind_speed,venue_surface,venue_roof_type,period,event_id,team_sequence,play_id,home_points,away_points,description,start_play_clock,start_play_clock_string,start_play_down,start_play_yfd,start_play_yardline,effective_start_play_yardline,start_play_field_side,inside_20,goaltogo,end_play_clock,end_play_clock_string,end_play_down,end_play_yfd,end_play_yardline,effective_end_play_yardline,end_play_field_side,firstdown,scoring_play,players_rushed,men_in_box,huddle,hash_mark,qb_at_snap,left_tightends,right_tightends,qb_name,pocket_location,play_direction,screen_pass,play_action,run_pass_option,pass_route,fake_punt,fake_field_goal,defender_name,player_name,player_jersey,player_position,player_sr_id,team_alias,team_sr_id,blitz,hurry,knockdown,pocket_time,pocket_time_string,on_target_throw,batted_pass,incompletion_type,target,reception,yards,att_yards,yards_after_catch,yards_after_contact,broken_tackles,dropped,catchable,touchdown
0,7d06369a-382a-448a-b295-6da9eab53245,2021,REG,3,MIN,sr:competitor:4423,SEA,sr:competitor:4430,77,44,7,artificial,dome,1,01ebde76-2065-4995-8871-5156d235c3fd,1,f72f3120-1f07-11ec-a67f-c75e589fa3a4,0,0,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.,0 days 00:15:00,15:00,1,10,25,75,SEA,0,0,0 days 00:14:24,14:24,1,10,42,58,SEA,1,NaN,4,7,Huddle,Middle,Under Center,0,2,Russell Wilson,Boot Right,Right,False,True,False,Cross,False,False,NaN,DK Metcalf,14,WR,sr:player:1230010,SEA,sr:competitor:4430,0,1,0,3,3.13,1,0,NaN,1,1,17,11,6,NaN,NaN,0,0,NaN
1,7d06369a-382a-448a-b295-6da9eab53245,2021,REG,3,MIN,sr:competitor:4423,SEA,sr:competitor:4430,77,44,7,artificial,dome,1,01ebde76-2065-4995-8871-5156d235c3fd,1,4c3a6ea0-1f08-11ec-a67f-c75e589fa3a4,0,0,R.Wilson pass short right complete to MIN 42. Catch made by C.Carson at MIN 42. Gain of 5 yards. Tackled by N.Vigil at MIN 40.,0 days 00:12:50,12:50,1,10,45,45,MIN,0,0,0 days 00:12:02,12:02,2,5,40,40,MIN,0,NaN,4,7,No Huddle,Right Hash,Shotgun,0,1,Russell Wilson,Middle,Right,False,True,True,Curl,False,False,NaN,Chris Carson,32,RB,sr:player:1130199,SEA,sr:competitor:4430,0,0,0,3,3.08,1,0,NaN,1,1,5,3,2,0,NaN,0,0,NaN
2,7d06369a-382a-448a-b295-6da9eab53245,2021,REG,3,MIN,sr:competitor:4423,SEA,sr:competitor:4430,77,44,7,artificial,dome,1,01ebde76-2065-4995-8871-5156d235c3fd,1,8e3944c0-1f08-11ec-a67f-c75e589fa3a4,0,0,R.Wilson pass short left complete to MIN 27. Catch made by D.Metcalf at MIN 27. Gain of 28 yards. Tackled by H.Smith at MIN 10.,0 days 00:11:24,11:24,3,3,38,38,MIN,0,0,0 days 00:10:41,10:41,1,10,10,10,MIN,1,NaN,4,7,No Huddle,Right Hash,Shotgun,0,1,Russell Wilson,Middle,Left,False,False,False,Post,False,False,NaN,DK Metcalf,14,WR,sr:player:1230010,SEA,sr:competitor:4430,0,0,0,3,3.21,1,0,NaN,1,1,28,11,17,2,NaN,0,0,NaN
3,7d06369a-382a-448a-b295-6da9eab53245,2021,REG,3,MIN,sr:competitor:4423,SEA,sr:competitor:4430,77,44,7,artificial,dome,1,01ebde76-2065-4995-8871-5156d235c3fd,1,b0cb36b0-1f08-11ec-a67f-c75e589fa3a4,0,6,"R.Wilson pass short right complete to MIN 3. Catch made by D.Metcalf at MIN 3. Gain of 10 yards. D.Metcalf for 10 yards, TOUCHDOWN.",0 days 00:10:32,10:32,2,10,10,10,MIN,1,1,0 days 00:10:26,10:26,0,0,15,15,MIN,1,True,4,7,Huddle,Right Hash,Shotgun,0,1,Russell Wilson,Middle,Right Sideline,False,False,False,Out,False,False,NaN,DK Metcalf,14,WR,sr:player:1230010,SEA,sr:competitor:4430,0,0,0,2,1.74,1,0,NaN,1,1,10,7,3,1,1,0,0,1
4,7d06369a-382a-448a-b295-6da9eab53245,2021,REG,3,MIN,sr:competitor:4423,SEA,sr:competitor:4430,77,44,7,artificial,dome,1,b67eb032-3253-4577-9387-8461866cdec0,1,86a1cc40-1f09-11ec-a67f-c75e589fa3a4,0,7,K.Cousins pass short left complete to MIN 35. Catch made by A.Mattison at MIN 35. Gain of 20 yards. Tackled by J.Adams at SEA 43.,0 days 00:09:51,9:51,2,3,37,63,MIN,0,0,0 days 00:09:11,9:11,1,10,43,43,SEA,1,NaN,4,8,Huddle,Middle,Under Center,0,1,Kirk Cousins,Middle,Le

### Fixing receiver features

#### Poorly thrown passes
- If the pass is poorly thrown, it should not be considered on target or catchable

In [70]:
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 1) & (sportradar_wr['incompletion_type'] == 'Poorly Thrown')][['play_id', 'period', 'start_play_clock_string', 'player_name', 'on_target_throw', 'catchable', 'incompletion_type', 'description']]

,play_id,period,start_play_clock_string,player_name,on_target_throw,catchable,incompletion_type,description
42,ec090b40-1f16-11ec-a9c1-1b4016cb3a46,3,7:28,Justin Jefferson,1,0,Poorly Thrown,K.Cousins steps back to pass. K.Cousins pass incomplete short right intended for J.Jefferson.
54,dffcfdc0-1f1b-11ec-a9c1-1b4016cb3a46,4,11:52,Freddie Swain,1,1,Poorly Thrown,R.Wilson steps back to pass. R.Wilson pass incomplete short middle intended for F.Swain.
65,7a5aac60-1f20-11ec-a9c1-1b4016cb3a46,4,00:13,DK Metcalf,1,0,Poorly Thrown,R.Wilson steps back to pass. R.Wilson pass incomplete short left intended for D.Metcalf.


In [71]:
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 1) & (sportradar_wr['incompletion_type'] == 'Poorly Thrown'), 'on_target_throw'] = 0

In [72]:
sportradar_wr.loc[(sportradar_wr['catchable'] == 1) & (sportradar_wr['incompletion_type'] == 'Poorly Thrown'), 'catchable'] = 0

#### Dropped passes
- Change `on_target_throw` to 1
- Change `catchable` to 1

In [73]:
sportradar_wr.loc[sportradar_wr['incompletion_type'] == 'Dropped Pass'][['play_id', 'period', 'start_play_clock_string', 'player_name', 'on_target_throw', 'catchable', 'incompletion_type', 'description']]

,play_id,period,start_play_clock_string,player_name,on_target_throw,catchable,incompletion_type,description
29,976035e0-1f12-11ec-acc0-f76b8dd971ea,2,00:38,Justin Jefferson,1,1,Dropped Pass,K.Cousins steps back to pass. K.Cousins pass incomplete short right intended for J.Jefferson.


In [74]:
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 0) & (sportradar_wr['incompletion_type'] == 'Dropped Pass'), 'on_target_throw'] = 1

In [75]:
sportradar_wr.loc[(sportradar_wr['catchable'] == 0) & (sportradar_wr['incompletion_type'] == 'Dropped Pass'), 'catchable'] = 1

#### Defended passes
- If pass was not batted...
  - Change `on_target_throw` to 1
  - Change `catchable` to 1

In [76]:
sportradar_wr.loc[sportradar_wr['incompletion_type'] == 'Pass Defended'][['play_id', 'period', 'start_play_clock_string', 'player_name', 'on_target_throw', 'catchable', 'incompletion_type', 'description']]

,play_id,period,start_play_clock_string,player_name,on_target_throw,catchable,incompletion_type,description
14,add74e40-1f0c-11ec-89e9-8992a3cbf855,2,14:56,Adam Thielen,0,0,Pass Defended,K.Cousins steps back to pass. K.Cousins pass incomplete short right intended for A.Thielen (C.Dunlap).
60,9f6be110-1f1e-11ec-a9c1-1b4016cb3a46,4,2:46,DK Metcalf,1,1,Pass Defended,R.Wilson steps back to pass. R.Wilson pass incomplete deep left intended for D.Metcalf (X.Woods).
61,bceea880-1f1e-11ec-a9c1-1b4016cb3a46,4,2:36,Penny Hart,1,1,Pass Defended,R.Wilson steps back to pass. R.Wilson pass incomplete deep left intended for P.Hart (H.Smith).


In [77]:
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 0) & (sportradar_wr['incompletion_type'] == 'Pass Defended') & (sportradar_wr['batted_pass'] == False), 'on_target_throw'] = 1

In [78]:
sportradar_wr.loc[(sportradar_wr['catchable'] == 0) & (sportradar_wr['incompletion_type'] == 'Pass Defended') & (sportradar_wr['batted_pass'] == False), 'catchable'] = 1

#### Caught passes

In [79]:
sportradar_wr.loc[(sportradar_wr['catchable'] == 0) & (sportradar_wr['reception'] == 1)][['play_id', 'period', 'start_play_clock_string', 'player_name', 'on_target_throw', 'catchable', 'incompletion_type', 'description']]

,play_id,period,start_play_clock_string,player_name,on_target_throw,catchable,incompletion_type,description
0,f72f3120-1f07-11ec-a67f-c75e589fa3a4,1,15:00,DK Metcalf,1,0,NaN,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.
1,4c3a6ea0-1f08-11ec-a67f-c75e589fa3a4,1,12:50,Chris Carson,1,0,NaN,R.Wilson pass short right complete to MIN 42. Catch made by C.Carson at MIN 42. Gain of 5 yards. Tackled by N.Vigil at MIN 40.
2,8e3944c0-1f08-11ec-a67f-c75e589fa3a4,1,11:24,DK Metcalf,1,0,NaN,R.Wilson pass short left complete to MIN 27. Catch made by D.Metcalf at MIN 27. Gain of 28 yards. Tackled by H.Smith at MIN 10.
3,b0cb36b0-1f08-11ec-a67f-c75e589fa3a4,1,10:32,DK Metcalf,1,0,NaN,"R.Wilson pass short right complete to MIN 3. Catch made by D.Metcalf at MIN 3. Gain of 10 yards. D.Metcalf for 10 yards, TOUCHDOWN."
4,86a1cc40-1f09-11ec-a67f-c75e589fa3a4,1,9:51,Alexander Mattison,1,0,NaN,K.Cousins pass short left complete to MIN 35. Catch made by A.Mattison at MIN 35. Gain of 20 yards. Tackled by J.Adams at SEA 43.
5,9e800700-1f09-11ec-a67f-c75e589fa3a4,1,9:11,Tyler Conklin,1,0,NaN,K.Cousins pass short left complete to SEA 32. Catch made by T.Conklin at SEA 32. Gain of 17 yards. Tackled by T.Flowers at SEA 26.
6,cf6a9c40-1f09-11ec-b36d-a3b2acec3de6,1,8:32,Justin Jefferson,1,0,NaN,"K.Cousins pass short left complete to SEA 27. Catch made by J.Jefferson at SEA 27. Gain of 8 yards. Tackled by T.Flowers, J.Adams at SEA 18."
7,dabe15e0-1f09-11ec-b36d-a3b2acec3de6,1,7:54,Justin Jefferson,1,0,NaN,K.Cousins pass short left complete to SEA 13. Catch made by J.Jefferson at SEA 13. Gain of 10 yards. Tackled by T.Flowers at SEA 8.
8,f6cb0630-1f09-11ec-b36d-a3b2acec3de6,1,6:42,Tyler Conklin,1,0,NaN,"K.Cousins pass complete to SEA 1. Catch made by T.Conklin at SEA 1. Gain of 7 yards. T.Conklin for 7 yards, TOUCHDOWN."
9,ca06a590-1f0a-11ec-b36d-a3b2acec3de6,1,5:14,Freddie Swain,1,0,NaN,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.


## New receiver columns
- We should create new features that capture the nuances of player style based on:
  - Utilization by route run
  - Physical attributes
  - Reliability in clutch situations
  - Contact tolerance
  - The ability to create separation from a defender

### Catches of poorly thrown passes
- To capture when a receiver bails out a quarterback by managing to catch an off-target pass
  - Demonstrates a receiver's overall "catch radius"
    - A combination of body control, height, arm length, hand size, and ball tracking ability
- Seems to only be recorded when a penalty is called.
  - Not sure if this dataset actually records catches of poorly thrown passes or assumes they don't get caught by default.
  - Still going to keep this feature just in case.

In [80]:
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 0) & (sportradar_wr['reception'] == 1) & (~sportradar_wr['description'].str.contains('penalty', case = False, na = False))][['play_id', 'period', 'start_play_clock_string', 'player_name', 'on_target_throw', 'catchable', 'incompletion_type', 'description']]

,play_id,period,start_play_clock_string,player_name,on_target_throw,catchable,incompletion_type,description


In [81]:
sportradar_wr['difficult_attempt'] = False
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 0) & (~sportradar_wr['description'].str.contains('penalty', case = False, na = False)), 'difficult_attempt'] = True

In [82]:
sportradar_wr['difficult_catch'] = False
sportradar_wr.loc[(sportradar_wr['on_target_throw'] == 0) & (sportradar_wr['reception'] == 1) & (~sportradar_wr['description'].str.contains('penalty', case = False, na = False)), 'difficult_catch'] = True

### Adverse weather catches
- Can the receiver keep his footing when there's heavy rain? Can he catch a wet ball?
- If OR:
  - Temp is 32 F or less
  - Wind speed is 10 MPH or greater
  - Humidity is 70% or greater

In [83]:
sportradar_wr.loc[(sportradar_wr['temp'] <= 32) | (sportradar_wr['wind_speed'] >= 10) | (sportradar_wr['humidity'] >= 70 )][['play_id', 'period', 'start_play_clock_string', 'player_name', 'on_target_throw', 'catchable', 'incompletion_type', 'description']]

,play_id,period,start_play_clock_string,player_name,on_target_throw,catchable,incompletion_type,description


In [84]:
sportradar_wr['weather_attempt'] = False
sportradar_wr.loc[((sportradar_wr['temp'] <= 32) | (sportradar_wr['wind_speed'] >= 10) | (sportradar_wr['humidity'] >= 70 )) & (sportradar_wr['reception'] == 1), 'weather_attempt'] = True

In [85]:
sportradar_wr['weather_catch'] = False
sportradar_wr.loc[((sportradar_wr['temp'] <= 32) | (sportradar_wr['wind_speed'] >= 10) | (sportradar_wr['humidity'] >= 70 )) & (sportradar_wr['reception'] == 1), 'weather_catch'] = True

### QB bail-out catches
- Does the receiver know when the QB is under duress, and does he know when and where to make himself open for a pass to prevent a loss of yardage?
- Catches made when the quarterback is under duress (if OR):
  - If QB scrambled
  - If QB was hurried
  - If QB was blitzed
  - If QB was knocked down

In [86]:
sportradar_wr.loc[(
                    (sportradar_wr['pocket_location'] == 'Scramble Left') 
                    | (sportradar_wr['pocket_location'] == 'Scramble Right') 
                    | (sportradar_wr['hurry'] == True) 
                    | (sportradar_wr['blitz'] == True) 
                    | (sportradar_wr['knockdown'] == True)
                  ) & (sportradar_wr['reception'] == 1)][['play_id', 'period', 'start_play_clock_string', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'description']]

,play_id,period,start_play_clock_string,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,description
0,f72f3120-1f07-11ec-a67f-c75e589fa3a4,1,15:00,Russell Wilson,Boot Right,1,0,0,4,3.13,Cross,DK Metcalf,11,17,6,1,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.
5,9e800700-1f09-11ec-a67f-c75e589fa3a4,1,9:11,Kirk Cousins,Boot Right,0,1,0,5,3.25,Cross,Tyler Conklin,11,17,6,1,K.Cousins pass short left complete to SEA 32. Catch made by T.Conklin at SEA 32. Gain of 17 yards. Tackled by T.Flowers at SEA 26.
7,dabe15e0-1f09-11ec-b36d-a3b2acec3de6,1,7:54,Kirk Cousins,Middle,0,1,0,5,1.78,Out,Justin Jefferson,5,10,5,1,K.Cousins pass short left complete to SEA 13. Catch made by J.Jefferson at SEA 13. Gain of 10 yards. Tackled by T.Flowers at SEA 8.
9,ca06a590-1f0a-11ec-b36d-a3b2acec3de6,1,5:14,Russell Wilson,Middle,0,1,0,6,1.59,Slant,Freddie Swain,8,10,2,1,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.
10,e3c698a0-1f0a-11ec-b36d-a3b2acec3de6,1,4:38,Russell Wilson,Middle,0,1,1,5,2.25,Post,DK Metcalf,12,12,0,1,R.Wilson pass short middle complete to SEA 50. Catch made by D.Metcalf at SEA 50. Gain of 12 yards. Tackled by B.Breeland at MIN 50.
21,19c341c0-1f0f-11ec-89e9-8992a3cbf855,2,5:32,Kirk Cousins,Middle,0,1,0,5,2.36,Post,Adam Thielen,15,15,0,1,"K.Cousins pass complete to SEA End Zone. Catch made by A.Thielen at SEA End Zone. Gain of 15 yards. A.Thielen for 15 yards, TOUCHDOWN."
23,0c9c7240-1f10-11ec-89e9-8992a3cbf855,2,4:41,Russell Wilson,Middle,0,1,0,5,2.46,Cross,Will Dissly,9,39,30,1,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.
28,7b2b7240-1f12-11ec-acc0-f76b8dd971ea,2,00:57,Kirk Cousins,Middle,0,1,0,5,2.25,Curl,Adam Thielen,7,7,0,1,K.Cousins pass complete to SEA 30. Catch made by A.Thielen at SEA 30. Gain of 7 yards. Tackled by U.Amadi at SEA 30.
37,17958820-1f16-11ec-a9c1-1b4016cb3a46,3,13:02,Kirk Cousins,Middle,0,1,1,6,2.75,Curl,Justin Jefferson,10,15,5,1,"K.Cousins pass short left complete to MIN 43. Catch made by J.Jefferson at MIN 43. Gain of 15 yards. Tackled by T.Flowers, U.Amadi at MIN 48."
38,362af810-1f16-11ec-a9c1-1b4016cb3a46,3,12:05,Kirk Cousins,Scramble Left,1,0,0,4,2.60,Flat,Tyler Conklin,1,5,4,1,K.Cousins pass short left complete to SEA 47. Catch made by T.Conklin at SEA 47. Gain of 5 yards. Tackled by U.Amadi at SEA 43.


In [87]:
sportradar_wr['qb_bf_attempt'] = False
sportradar_wr.loc[(
                    (sportradar_wr['pocket_location'] == 'Scramble Left') 
                    | (sportradar_wr['pocket_location'] == 'Scramble Right') 
                    | (sportradar_wr['hurry'] == True) 
                    | (sportradar_wr['blitz'] == True) 
                    | (sportradar_wr['knockdown'] == True)
                  ) , 'qb_bf_attempt'] = True

sportradar_wr['qb_bf_catch'] = False
sportradar_wr.loc[(
                    (sportradar_wr['pocket_location'] == 'Scramble Left') 
                    | (sportradar_wr['pocket_location'] == 'Scramble Right') 
                    | (sportradar_wr['hurry'] == True) 
                    | (sportradar_wr['blitz'] == True) 
                    | (sportradar_wr['knockdown'] == True)
                  ) 
                  & (sportradar_wr['reception'] == 1), 'qb_bf_catch'] = True

### Clutch catches
- Does the receiver sustain drives by converting first downs or catching touchdowns near the end of close games? Or does he buckle under the pressure?
- If AND:
  - 4th quarter or overtime AND 4 or fewer minutes left in period
  - Difference of one score (8 or fewer points)
  - Converts first down or is touchdown

In [88]:
sportradar_wr.loc[(
                    (sportradar_wr['period'] >= 4) & (sportradar_wr['start_play_clock'] < pd.Timedelta(minutes=4)) 
                    & (
                        (
                            (sportradar_wr['team_alias'] == sportradar_wr['home_alias']) & (abs(sportradar_wr['away_points'] - sportradar_wr['home_points']) <= 8)
                        )
                      | (
                            (sportradar_wr['team_alias'] == sportradar_wr['away_alias']) & (abs(sportradar_wr['home_points'] - sportradar_wr['away_points']) <= 8)
                        )
                      )
                    & (
                        (sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd'])
                      )
                  ) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'qb_name', 'pocket_location', 'firstdown', 'goaltogo',
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,qb_name,pocket_location,firstdown,goaltogo,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,description


In [89]:
sportradar_wr.columns

Index(['game_id', 'season_year', 'season_type', 'week', 'home_alias',
       'home_sr_id', 'away_alias', 'away_sr_id', 'temp', 'humidity',
       'wind_speed', 'venue_surface', 'venue_roof_type', 'period', 'event_id',
       'team_sequence', 'play_id', 'home_points', 'away_points', 'description',
       'start_play_clock', 'start_play_clock_string', 'start_play_down',
       'start_play_yfd', 'start_play_yardline',
       'effective_start_play_yardline', 'start_play_field_side', 'inside_20',
       'goaltogo', 'end_play_clock', 'end_play_clock_string', 'end_play_down',
       'end_play_yfd', 'end_play_yardline', 'effective_end_play_yardline',
       'end_play_field_side', 'firstdown', 'scoring_play', 'players_rushed',
       'men_in_box', 'huddle', 'hash_mark', 'qb_at_snap', 'left_tightends',
       'right_tightends', 'qb_name', 'pocket_location', 'play_direction',
       'screen_pass', 'play_action', 'run_pass_option', 'pass_route',
       'fake_punt', 'fake_field_goal', 'defender_nam

In [90]:
sportradar_wr['clutch_catch'] = False
sportradar_wr.loc[(
                    (sportradar_wr['period'] >= 4) & (sportradar_wr['start_play_clock'] < pd.Timedelta(minutes=4)) 
                    & (
                        (
                            (sportradar_wr['team_alias'] == sportradar_wr['home_alias']) & (abs(sportradar_wr['away_points'] - sportradar_wr['home_points']) <= 8)
                        )
                      | (
                            (sportradar_wr['team_alias'] == sportradar_wr['away_alias']) & (abs(sportradar_wr['home_points'] - sportradar_wr['away_points']) <= 8)
                        )
                      )
                    & (
                        (sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd'])
                      )
                  ) & (sportradar_wr['reception'] == 1), 'clutch_catch'
                ] = True

In [91]:
sportradar_wr[(sportradar_wr['goaltogo'] == 1) & (sportradar_wr['start_play_yardline'] >= 10)][['period', 'start_play_clock', 'start_play_down', 'inside_20', 'start_play_yfd', 'goaltogo', 'start_play_yardline', 'att_yards', 'description']]

,period,start_play_clock,start_play_down,inside_20,start_play_yfd,goaltogo,start_play_yardline,att_yards,description
3,1,0 days 00:10:32,2,1,10,1,10,7,"R.Wilson pass short right complete to MIN 3. Catch made by D.Metcalf at MIN 3. Gain of 10 yards. D.Metcalf for 10 yards, TOUCHDOWN."


### Conversion catches
- Does the receiver sustain drives or catch touchdowns when the drive is in jeopardy?
- The offensive team's 3rd down is commonly regarded as the last practical down of a drive
  - Although the offensive team has 4 downs to sustain a drive or score, the 4th down is commonly used to kick the ball in one of two ways:
    - Kicking a field goal for 3 points, provided that the offensive team is within the placekicker's range
    - Punting the ball to move it downfield so that the opposing team starts with worse field position when it receives the punt
  - If the offensive team attempts a pass or run play on 4th down and fails to convert a 1st down or score, the referee declares a "turnover on downs" and the opposing team takes possession of the ball where it was downed on the failed conversion attempt
    - Due to the difficulty, pressure, and cost of converting on 4th down, offensive teams typically use the 4th down to punt or kick a field goal
- If OR:
  - 3rd down or greater and converts 1st down
  - 3rd down or greater and scores TD

In [92]:
sportradar_wr.loc[(
                    (
                        ((sportradar_wr['start_play_down'] >= 3) & (sportradar_wr['firstdown'] == 1)) | ((sportradar_wr['start_play_down'] >= 3) & (sportradar_wr['touchdown'] == 1))
                    )
                  ) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'qb_name', 'pocket_location', 'goaltogo',
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,qb_name,pocket_location,goaltogo,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,firstdown,touchdown,description
2,8e3944c0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,11:24,Russell Wilson,Middle,0,0,0,0,4,3.21,Post,DK Metcalf,11,28,17,1,1,NaN,R.Wilson pass short left complete to MIN 27. Catch made by D.Metcalf at MIN 27. Gain of 28 yards. Tackled by H.Smith at MIN 10.
9,ca06a590-1f0a-11ec-b36d-a3b2acec3de6,7,7,1,5:14,Russell Wilson,Middle,0,0,1,0,6,1.59,Slant,Freddie Swain,8,10,2,1,1,NaN,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.
21,19c341c0-1f0f-11ec-89e9-8992a3cbf855,13,17,2,5:32,Kirk Cousins,Middle,0,0,1,0,5,2.36,Post,Adam Thielen,15,15,0,1,1,1,"K.Cousins pass complete to SEA End Zone. Catch made by A.Thielen at SEA End Zone. Gain of 15 yards. A.Thielen for 15 yards, TOUCHDOWN."
28,7b2b7240-1f12-11ec-acc0-f76b8dd971ea,14,17,2,00:57,Kirk Cousins,Middle,0,0,1,0,5,2.25,Curl,Adam Thielen,7,7,0,1,1,NaN,K.Cousins pass complete to SEA 30. Catch made by A.Thielen at SEA 30. Gain of 7 yards. Tackled by U.Amadi at SEA 30.
30,c448f1f0-1f12-11ec-acc0-f76b8dd971ea,14,17,2,00:29,Kirk Cousins,Middle,0,0,0,0,4,1.78,Slant,Justin Jefferson,9,9,0,1,1,NaN,K.Cousins pass short middle complete to SEA 19. Catch made by J.Jefferson at SEA 19. Gain of 9 yards. Tackled by U.Amadi at SEA 19.
40,8e79a0c0-1f16-11ec-a9c1-1b4016cb3a46,21,17,3,10:03,Kirk Cousins,Middle,0,0,0,0,4,1.89,Out,Justin Jefferson,8,8,0,1,1,NaN,K.Cousins pass complete to SEA 26. Catch made by J.Jefferson at SEA 26. Gain of 8 yards. ran out of bounds.
47,f896a0a0-1f18-11ec-a9c1-1b4016cb3a46,24,17,3,1:40,Kirk Cousins,Middle,0,0,0,0,4,1.87,Out,K.J. Osborn,6,11,5,1,1,NaN,"K.Cousins pass short right complete to MIN 46. Catch made by K.Osborn at MIN 46. Gain of 11 yards. Pushed out of bounds by Q.Diggs, J.Brooks at SEA 49."
56,e07f57b0-1f1c-11ec-a9c1-1b4016cb3a46,27,17,4,8:21,Kirk Cousins,Middle,0,0,1,1,6,1.11,Cross,K.J. Osborn,5,15,10,1,1,NaN,K.Cousins pass short left complete to SEA 32. Catch made by K.Osborn at SEA 32. Gain of 15 yards. Tackled by U.Amadi at SEA 22.
62,4b97a780-1f1f-11ec-a9c1-1b4016cb3a46,30,17,4,2:08,Kirk Cousins,Middle,0,0,0,0,4,2.16,Out,Justin Jefferson,11,11,0,1,1,NaN,K.Cousins pass short right complete to SEA 44. Catch made by J.Jefferson at SEA 44. Gain of 11 yards. J.Jefferson ran out of bounds.


In [93]:
sportradar_wr['conversion_attempt'] = False
sportradar_wr.loc[
                    (sportradar_wr['start_play_down'] >= 3) & (sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd']),
                    'conversion_attempt'
                ] = True

sportradar_wr['conversion_catch'] = False
sportradar_wr.loc[
                    (sportradar_wr['start_play_down'] >= 3) & ((sportradar_wr['firstdown'] == 1) | (sportradar_wr['touchdown'] == 1)) & (sportradar_wr['reception'] == 1),
                    'conversion_catch'
                ] = True

### Red zone touchdown catches
- Touchdown catches from the 20 yardline or closer
- Completing a pass in the red zone is difficult because the play starts from within the 20 yardline, making the 22 players bunched very closely together
  - There is much less room for receivers to create horizontal separation from defenders
  - Taller receivers are favored on redzone pass plays because they can use their height, arm length, and jumping ability to create separation

In [94]:
sportradar_wr.loc[(
                    (
                        (sportradar_wr['inside_20'] == 1) & (sportradar_wr['touchdown'] == 1)
                    )
                  ) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,firstdown,touchdown,description
3,b0cb36b0-1f08-11ec-a67f-c75e589fa3a4,0,6,1,10:32,2,10,1,1,Russell Wilson,Middle,0,0,0,4,1.74,Out,DK Metcalf,7,10,3,1,1,1,"R.Wilson pass short right complete to MIN 3. Catch made by D.Metcalf at MIN 3. Gain of 10 yards. D.Metcalf for 10 yards, TOUCHDOWN."
8,f6cb0630-1f09-11ec-b36d-a3b2acec3de6,6,7,1,6:42,2,7,1,1,Kirk Cousins,Middle,0,0,0,3,2.15,Out,Tyler Conklin,6,7,1,1,1,1,"K.Cousins pass complete to SEA 1. Catch made by T.Conklin at SEA 1. Gain of 7 yards. T.Conklin for 7 yards, TOUCHDOWN."
21,19c341c0-1f0f-11ec-89e9-8992a3cbf855,13,17,2,5:32,3,15,1,0,Kirk Cousins,Middle,0,1,0,5,2.36,Post,Adam Thielen,15,15,0,1,1,1,"K.Cousins pass complete to SEA End Zone. Catch made by A.Thielen at SEA End Zone. Gain of 15 yards. A.Thielen for 15 yards, TOUCHDOWN."
32,0a490500-1f13-11ec-acc0-f76b8dd971ea,20,17,2,00:20,1,3,1,1,Kirk Cousins,Middle,0,0,0,4,2.30,Out,Justin Jefferson,3,3,0,1,1,1,"K.Cousins pass complete to SEA End Zone. Catch made by J.Jefferson at SEA End Zone. Gain of 3 yards. J.Jefferson for 3 yards, TOUCHDOWN."


In [95]:
sportradar_wr['redzone_attempt'] = False
sportradar_wr.loc[(
                    (sportradar_wr['inside_20'] == 1) & (sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd'])
                  ), 'redzone_attempt'] = True

sportradar_wr['redzone_catch'] = False
sportradar_wr.loc[(
                    (sportradar_wr['inside_20'] == 1) & (sportradar_wr['touchdown'] == 1)
                  )
                   & (sportradar_wr['reception'] == 1), 'redzone_catch'] = True

### Catches by route
- Elite route runners are able to change direction very quickly without telegraphing their intentions to their defenders
- Routes that feature quick direction change, such as the curl, require that the receiver have these traits:
- Rapid acceleration
- Rapid deceleration - The ability to stop movement in one direction with very few steps
  - If a receiver "pitter-patters," the defender knows he intends to stop and change directions
- Flexible ankles
- Hip mobility
- Leverage intuition
  - "Inside leverage" - When a receiver is positioned with his defender closer to the nearest lateral boundary, making the receiver closer to the "inside" of the field
  - "Outside leverage" - When a receiver is positioned closer to the nearest lateral boundary than his defender, making the receiver closer to the "outside" of the field
  - Understanding leverage tells a receiver:
    - When to expect the pass without looking back at the QB
    - What kind of pass the QB will throw
- Not every elite receiver is an elite route runner.
  - Some can run a limited "route tree," running only relatively straight routes that take advantage of their speed.

In [96]:
routes = ['In', 'Slant', 'Corner', 'Flat', 'Curl', 'WR Screen', 'Out', 'Go', 'Cross', 'Post', 'Comeback', 'Underneath Screen']

for r in routes:
    sportradar_wr[r.replace(' ', '_').lower() + '_attempt'] = False
    sportradar_wr.loc[ (sportradar_wr['pass_route'] == r), r.replace(' ', '_').lower() + '_attempt'] = True

for r in routes:
    sportradar_wr[r.replace(' ', '_').lower()+'_catch'] = False
    sportradar_wr.loc[ (sportradar_wr['pass_route'] == r) & (sportradar_wr['reception'] == 1), r.replace(' ', '_').lower()+'_catch'] = True

In [97]:
# routes = ['Cross', 'Curl', 'Post', 'Underneath Screen', 'Flat', 'Slant', 'WR Screen', 'Comeback', 'Go', 'In']

# for r in routes:
#     sportradar_wr[r.replace(' ', '_').lower()+'_catch'] = False
#     sportradar_wr.loc[ (sportradar_wr['pass_route'] == r) & (sportradar_wr['reception'] == 1), r.replace(' ', '_').lower()+'_catch'] = True

### Deep catch
- A pass play can result in a large gain, but this can be achieved by a short pass to a receiver who gains many yards after the catch
- This metric is meant to capture receivers who are frequently relied on to catch deep passes - passes that travel 20+ yards through the air before a catch attempt is made

In [98]:
sportradar_wr.loc[ (sportradar_wr['att_yards'] >= 20) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,firstdown,touchdown,description
15,128b2eb0-1f0d-11ec-89e9-8992a3cbf855,7,10,2,14:09,1,35,0,0,Russell Wilson,Middle,0,0,0,4,2.58,Comeback,DK Metcalf,20,21,1,1,1,NaN,R.Wilson pass deep left complete to MIN 45. Catch made by D.Metcalf at MIN 45. Gain of 21 yards. Pushed out of bounds by X.Woods at MIN 44.


In [99]:
sportradar_wr['deep_attempt'] = False
sportradar_wr.loc[ (sportradar_wr['att_yards'] >= 20), 'deep_attempt'] = True

In [100]:
sportradar_wr['deep_catch'] = False
sportradar_wr.loc[ (sportradar_wr['att_yards'] >= 20) & (sportradar_wr['reception'] == 1), 'deep_catch'] = True

### Large YAC catch
- Some receivers, such as Deebo Samuel, excel at catching short passes and gaining many yards after the catch
  - This requires great vision, lateral agility, and acceleration
- Catches with 10+ yards after catch

In [101]:
sportradar_wr.loc[ (sportradar_wr['yards_after_catch'] >= 10) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,firstdown,touchdown,description
2,8e3944c0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,11:24,3,38,0,0,Russell Wilson,Middle,0,0,0,4,3.21,Post,DK Metcalf,11,28,17,1,1,NaN,R.Wilson pass short left complete to MIN 27. Catch made by D.Metcalf at MIN 27. Gain of 28 yards. Tackled by H.Smith at MIN 10.
4,86a1cc40-1f09-11ec-a67f-c75e589fa3a4,0,7,1,9:51,2,37,0,0,Kirk Cousins,Middle,0,0,0,4,2.57,Underneath Screen,Alexander Mattison,-2,20,22,1,1,NaN,K.Cousins pass short left complete to MIN 35. Catch made by A.Mattison at MIN 35. Gain of 20 yards. Tackled by J.Adams at SEA 43.
11,424963d0-1f0b-11ec-b36d-a3b2acec3de6,7,7,1,2:04,2,33,0,0,Russell Wilson,Middle,0,0,0,4,2.47,Underneath Screen,Tyler Lockett,-7,3,10,1,0,NaN,"R.Wilson pass short left complete to MIN 40. Catch made by T.Lockett at MIN 40. Gain of 3 yards. Tackled by H.Smith at MIN 30. PENALTY on SEA-A.Collins, Offensive Illegal Block Above the Waist, 10 yards, accepted."
17,63a98520-1f0e-11ec-89e9-8992a3cbf855,7,17,2,9:40,1,27,0,0,Kirk Cousins,Middle,0,0,0,4,2.24,Underneath Screen,Alexander Mattison,-3,23,26,1,1,NaN,K.Cousins pass short right complete to MIN 24. Catch made by A.Mattison at MIN 24. Gain of 23 yards. Tackled by U.Amadi at MIN 50.
18,b25ceb80-1f0e-11ec-89e9-8992a3cbf855,7,17,2,7:25,1,38,0,0,Kirk Cousins,Middle,0,0,0,4,2.56,Post,Justin Jefferson,13,26,13,1,1,NaN,K.Cousins pass short middle complete to SEA 25. Catch made by J.Jefferson at SEA 25. Gain of 26 yards. Tackled by D.Reed at SEA 12.
22,fbbeaab0-1f0f-11ec-89e9-8992a3cbf855,14,17,2,5:23,1,21,0,0,Russell Wilson,Middle,0,0,0,4,0.22,Flat,Penny Hart,-4,7,11,1,0,NaN,R.Wilson pass short middle complete to SEA 17. Catch made by P.Hart at SEA 17. Gain of 7 yards. Tackled by H.Smith at SEA 28.
23,0c9c7240-1f10-11ec-89e9-8992a3cbf855,14,17,2,4:41,2,28,0,0,Russell Wilson,Middle,0,1,0,5,2.46,Cross,Will Dissly,9,39,30,1,1,NaN,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.
26,712b9730-1f11-11ec-acc0-f76b8dd971ea,14,17,2,3:04,2,36,0,0,Kirk Cousins,Middle,0,0,0,3,2.02,Underneath Screen,Alexander Mattison,-3,7,10,1,0,NaN,"K.Cousins pass short left complete to MIN 33. Catch made by A.Mattison at MIN 33. Gain of 7 yards. Tackled by J.Adams, P.Ford at MIN 43."
31,db457f40-1f12-11ec-acc0-f76b8dd971ea,14,17,2,00:24,1,19,1,0,Kirk Cousins,Middle,0,0,0,4,1.52,Slant,Tyler Conklin,3,16,13,1,1,NaN,K.Cousins pass short right complete to SEA 16. Catch made by T.Conklin at SEA 16. Gain of 16 yards. Tackled by J.Adams at SEA 3.
34,93c195e0-1f13-11ec-acc0-f76b8dd971ea,21,17,2,00:09,1,37,0,0,Russell Wilson,Middle,0,0,0,4,2.69,Flat,Gerald Everett,0,11,11,1,1,NaN,R.Wilson pass complete to SEA 37. Catch made by G.Everett at SEA 37. Gain of 11 yards. Pushed out of bounds by M.Alexander at SEA 48.


In [102]:
sportradar_wr['large_yac_catch'] = False
sportradar_wr.loc[ (sportradar_wr['yards_after_catch'] >= 10) & (sportradar_wr['reception'] == 1), 'large_yac_catch'] = True

### Play action catch
- A play action play is when the quarterback takes the snap from directly "under center" (right behind the center), runs backward, and fakes a hand-off to the running back
  - This play design fakes a run attempt, causing defenders to get "drawn up" leaving the deeper parts of the field more open for a receiver to exploit
- Catches made from play action

In [103]:
sportradar_wr.loc[ (sportradar_wr['play_action'] == True) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,firstdown,touchdown,description
0,f72f3120-1f07-11ec-a67f-c75e589fa3a4,0,0,1,15:00,1,25,0,0,Russell Wilson,Boot Right,1,0,0,4,3.13,Cross,DK Metcalf,11,17,6,1,1,NaN,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.
1,4c3a6ea0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,12:50,1,45,0,0,Russell Wilson,Middle,0,0,0,4,3.08,Curl,Chris Carson,3,5,2,1,0,NaN,R.Wilson pass short right complete to MIN 42. Catch made by C.Carson at MIN 42. Gain of 5 yards. Tackled by N.Vigil at MIN 40.
4,86a1cc40-1f09-11ec-a67f-c75e589fa3a4,0,7,1,9:51,2,37,0,0,Kirk Cousins,Middle,0,0,0,4,2.57,Underneath Screen,Alexander Mattison,-2,20,22,1,1,NaN,K.Cousins pass short left complete to MIN 35. Catch made by A.Mattison at MIN 35. Gain of 20 yards. Tackled by J.Adams at SEA 43.
5,9e800700-1f09-11ec-a67f-c75e589fa3a4,0,7,1,9:11,1,43,0,0,Kirk Cousins,Boot Right,0,1,0,5,3.25,Cross,Tyler Conklin,11,17,6,1,1,NaN,K.Cousins pass short left complete to SEA 32. Catch made by T.Conklin at SEA 32. Gain of 17 yards. Tackled by T.Flowers at SEA 26.
10,e3c698a0-1f0a-11ec-b36d-a3b2acec3de6,7,7,1,4:38,1,38,0,0,Russell Wilson,Middle,0,1,1,5,2.25,Post,DK Metcalf,12,12,0,1,1,NaN,R.Wilson pass short middle complete to SEA 50. Catch made by D.Metcalf at SEA 50. Gain of 12 yards. Tackled by B.Breeland at MIN 50.
11,424963d0-1f0b-11ec-b36d-a3b2acec3de6,7,7,1,2:04,2,33,0,0,Russell Wilson,Middle,0,0,0,4,2.47,Underneath Screen,Tyler Lockett,-7,3,10,1,0,NaN,"R.Wilson pass short left complete to MIN 40. Catch made by T.Lockett at MIN 40. Gain of 3 yards. Tackled by H.Smith at MIN 30. PENALTY on SEA-A.Collins, Offensive Illegal Block Above the Waist, 10 yards, accepted."
16,113a9d60-1f0e-11ec-89e9-8992a3cbf855,7,17,2,11:07,1,15,0,0,Kirk Cousins,Middle,0,0,0,4,2.92,Curl,Alexander Mattison,3,6,3,1,0,NaN,K.Cousins pass short middle complete to MIN 18. Catch made by A.Mattison at MIN 18. Gain of 6 yards. Tackled by J.Brooks at MIN 21.
23,0c9c7240-1f10-11ec-89e9-8992a3cbf855,14,17,2,4:41,2,28,0,0,Russell Wilson,Middle,0,1,0,5,2.46,Cross,Will Dissly,9,39,30,1,1,NaN,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.
32,0a490500-1f13-11ec-acc0-f76b8dd971ea,20,17,2,00:20,1,3,1,1,Kirk Cousins,Middle,0,0,0,4,2.30,Out,Justin Jefferson,3,3,0,1,1,1,"K.Cousins pass complete to SEA End Zone. Catch made by J.Jefferson at SEA End Zone. Gain of 3 yards. J.Jefferson for 3 yards, TOUCHDOWN."
38,362af810-1f16-11ec-a9c1-1b4016cb3a46,21,17,3,12:05,2,48,0,0,Kirk Cousins,Scramble Left,1,0,0,4,2.60,Flat,Tyler Conklin,1,5,4,1,0,NaN,K.Cousins pass short left complete to SEA 47. Catch made by T.Conklin at SEA 47. Gain of 5 yards. Tackled by U.Amadi at SEA 43.


In [104]:
sportradar_wr['play_action_attempt'] = False
sportradar_wr.loc[ (sportradar_wr['play_action'] == True), 'play_action_attempt'] = True

In [105]:
sportradar_wr['play_action_catch'] = False
sportradar_wr.loc[ (sportradar_wr['play_action'] == True) & (sportradar_wr['reception'] == 1), 'play_action_catch'] = True

### RPO catch
- A run-pass option is similar to a play action design in that it is a fake designed to deceive defenses, but the quarterback has multiple options.
- Unlike a play action concept, the QB does not turn his back toward the defense. He keeps his eyes downfield at all times. With his eyes downfield, he motions as if he's handing the ball off to the running back. He has 3 options:
  - Complete the hand-off to the running back
  - Keep the ball, thus faking the hand-off, and run
  - Keep the ball, thus faking the hand-off, and pass
- Catches made on a run-pass option

In [106]:
sportradar_wr.loc[ (sportradar_wr['run_pass_option'] == True) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,on_target_throw,firstdown,touchdown,description
1,4c3a6ea0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,12:50,1,45,0,0,Russell Wilson,Middle,0,0,0,4,3.08,Curl,Chris Carson,3,5,2,1,0,NaN,R.Wilson pass short right complete to MIN 42. Catch made by C.Carson at MIN 42. Gain of 5 yards. Tackled by N.Vigil at MIN 40.
32,0a490500-1f13-11ec-acc0-f76b8dd971ea,20,17,2,00:20,1,3,1,1,Kirk Cousins,Middle,0,0,0,4,2.30,Out,Justin Jefferson,3,3,0,1,1,1,"K.Cousins pass complete to SEA End Zone. Catch made by J.Jefferson at SEA End Zone. Gain of 3 yards. J.Jefferson for 3 yards, TOUCHDOWN."
38,362af810-1f16-11ec-a9c1-1b4016cb3a46,21,17,3,12:05,2,48,0,0,Kirk Cousins,Scramble Left,1,0,0,4,2.60,Flat,Tyler Conklin,1,5,4,1,0,NaN,K.Cousins pass short left complete to SEA 47. Catch made by T.Conklin at SEA 47. Gain of 5 yards. Tackled by U.Amadi at SEA 43.


In [107]:
sportradar_wr['rpo_attempt'] = False
sportradar_wr.loc[ (sportradar_wr['run_pass_option'] == True), 'rpo_attempt'] = True

In [108]:
sportradar_wr['rpo_catch'] = False
sportradar_wr.loc[ (sportradar_wr['run_pass_option'] == True) & (sportradar_wr['reception'] == 1), 'rpo_catch'] = True

### Tackle-breaker catch
- Some receives use their size and strength to bully defenders, inviting contact that fatigues defenders over the course of the game
- Tired defenders refer to the dilemma of tackling a physical ball carrier as a "business decision"
  - "Expend more energy and get hurt trying to tackle him, or let him run by me?"
- Catches after which the receiver breaks a tackle

In [109]:
sportradar_wr.loc[ (sportradar_wr['broken_tackles'] >= 1) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'yards_after_contact', 'broken_tackles', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,yards_after_contact,broken_tackles,on_target_throw,firstdown,touchdown,description
3,b0cb36b0-1f08-11ec-a67f-c75e589fa3a4,0,6,1,10:32,2,10,1,1,Russell Wilson,Middle,0,0,0,4,1.74,Out,DK Metcalf,7,10,3,1,1,1,1,1,"R.Wilson pass short right complete to MIN 3. Catch made by D.Metcalf at MIN 3. Gain of 10 yards. D.Metcalf for 10 yards, TOUCHDOWN."
12,7b593d30-1f0b-11ec-b36d-a3b2acec3de6,7,7,1,1:24,2,40,0,0,Russell Wilson,Middle,0,0,0,4,0.73,WR Screen,Gerald Everett,-3,5,8,3,1,1,0,NaN,R.Wilson pass short left complete to MIN 43. Catch made by G.Everett at MIN 43. Gain of 5 yards. Tackled by H.Smith at MIN 35.
17,63a98520-1f0e-11ec-89e9-8992a3cbf855,7,17,2,9:40,1,27,0,0,Kirk Cousins,Middle,0,0,0,4,2.24,Underneath Screen,Alexander Mattison,-3,23,26,6,1,1,1,NaN,K.Cousins pass short right complete to MIN 24. Catch made by A.Mattison at MIN 24. Gain of 23 yards. Tackled by U.Amadi at MIN 50.
23,0c9c7240-1f10-11ec-89e9-8992a3cbf855,14,17,2,4:41,2,28,0,0,Russell Wilson,Middle,0,1,0,5,2.46,Cross,Will Dissly,9,39,30,18,1,1,1,NaN,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.
34,93c195e0-1f13-11ec-acc0-f76b8dd971ea,21,17,2,00:09,1,37,0,0,Russell Wilson,Middle,0,0,0,4,2.69,Flat,Gerald Everett,0,11,11,4,1,1,1,NaN,R.Wilson pass complete to SEA 37. Catch made by G.Everett at SEA 37. Gain of 11 yards. Pushed out of bounds by M.Alexander at SEA 48.
35,b904c520-1f13-11ec-acc0-f76b8dd971ea,21,17,2,00:01,1,48,0,0,Russell Wilson,Middle,0,0,0,4,2.51,Flat,Travis Homer,-1,31,32,14,1,1,1,NaN,R.Wilson pass short left complete to SEA 47. Catch made by T.Homer at SEA 47. Gain of 31 yards. Tackled by C.Bynum at MIN 21.
51,132d0ba0-1f1b-11ec-a9c1-1b4016cb3a46,27,17,4,13:21,2,26,0,0,Russell Wilson,Middle,0,0,0,4,2.15,Curl,Gerald Everett,4,14,10,7,1,1,1,NaN,R.Wilson pass short right complete to SEA 30. Catch made by G.Everett at SEA 30. Gain of 14 yards. Tackled by N.Vigil at SEA 40.


In [110]:
sportradar_wr['tackle_breaker_catch'] = False
sportradar_wr.loc[ (sportradar_wr['broken_tackles'] >= 1) & (sportradar_wr['reception'] == 1), 'tackle_breaker_catch'] = True

### Beast catch
- Similar in intent to the tackle-breaker catch, but focuses on receivers who gain many yards after being contacted by a defender
- Catches that result in 10 or more yards after contact

In [111]:
sportradar_wr.loc[ (sportradar_wr['yards_after_contact'] >= 10) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 
                                                            'att_yards', 'yards', 'yards_after_catch', 'yards_after_contact', 'broken_tackles', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,yards_after_contact,broken_tackles,on_target_throw,firstdown,touchdown,description
23,0c9c7240-1f10-11ec-89e9-8992a3cbf855,14,17,2,4:41,2,28,0,0,Russell Wilson,Middle,0,1,0,5,2.46,Cross,Will Dissly,9,39,30,18,1,1,1,NaN,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.
35,b904c520-1f13-11ec-acc0-f76b8dd971ea,21,17,2,00:01,1,48,0,0,Russell Wilson,Middle,0,0,0,4,2.51,Flat,Travis Homer,-1,31,32,14,1,1,1,NaN,R.Wilson pass short left complete to SEA 47. Catch made by T.Homer at SEA 47. Gain of 31 yards. Tackled by C.Bynum at MIN 21.


In [112]:
sportradar_wr['beast_catch'] = False
sportradar_wr.loc[ (sportradar_wr['yards_after_contact'] >= 10) & (sportradar_wr['reception'] == 1), 'beast_catch'] = True

### Hurry-up catch
- Catches made on plays before which there was no huddle
- Measures a receiver's knowledge of the playbook and what their role is without detailed explanation
- Demonstrates receiver's chemistry with QB and whether they can be trusted to be ready at all times
- Crucial in time-limited situations

In [113]:
sportradar_wr.loc[ (sportradar_wr['huddle'] == 'No Huddle') & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                            'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 'att_yards', 'yards', 'yards_after_catch', 'yards_after_contact', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,yards_after_contact,on_target_throw,firstdown,touchdown,description
1,4c3a6ea0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,12:50,1,45,0,0,Russell Wilson,Middle,0,0,0,4,3.08,Curl,Chris Carson,3,5,2,0,1,0,NaN,R.Wilson pass short right complete to MIN 42. Catch made by C.Carson at MIN 42. Gain of 5 yards. Tackled by N.Vigil at MIN 40.
2,8e3944c0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,11:24,3,38,0,0,Russell Wilson,Middle,0,0,0,4,3.21,Post,DK Metcalf,11,28,17,2,1,1,NaN,R.Wilson pass short left complete to MIN 27. Catch made by D.Metcalf at MIN 27. Gain of 28 yards. Tackled by H.Smith at MIN 10.
9,ca06a590-1f0a-11ec-b36d-a3b2acec3de6,7,7,1,5:14,3,28,0,0,Russell Wilson,Middle,0,1,0,6,1.59,Slant,Freddie Swain,8,10,2,NaN,1,1,NaN,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.
15,128b2eb0-1f0d-11ec-89e9-8992a3cbf855,7,10,2,14:09,1,35,0,0,Russell Wilson,Middle,0,0,0,4,2.58,Comeback,DK Metcalf,20,21,1,NaN,1,1,NaN,R.Wilson pass deep left complete to MIN 45. Catch made by D.Metcalf at MIN 45. Gain of 21 yards. Pushed out of bounds by X.Woods at MIN 44.
27,54079c20-1f12-11ec-acc0-f76b8dd971ea,14,17,2,1:31,1,43,0,0,Kirk Cousins,Middle,0,0,0,4,2.23,Curl,Alexander Mattison,3,4,1,0,1,0,NaN,K.Cousins pass short middle complete to SEA 40. Catch made by A.Mattison at SEA 40. Gain of 4 yards. Tackled by B.Wagner; C.Barton at SEA 39.
38,362af810-1f16-11ec-a9c1-1b4016cb3a46,21,17,3,12:05,2,48,0,0,Kirk Cousins,Scramble Left,1,0,0,4,2.60,Flat,Tyler Conklin,1,5,4,2,1,0,NaN,K.Cousins pass short left complete to SEA 47. Catch made by T.Conklin at SEA 47. Gain of 5 yards. Tackled by U.Amadi at SEA 43.
51,132d0ba0-1f1b-11ec-a9c1-1b4016cb3a46,27,17,4,13:21,2,26,0,0,Russell Wilson,Middle,0,0,0,4,2.15,Curl,Gerald Everett,4,14,10,7,1,1,NaN,R.Wilson pass short right complete to SEA 30. Catch made by G.Everett at SEA 30. Gain of 14 yards. Tackled by N.Vigil at SEA 40.
52,2d901c80-1f1b-11ec-a9c1-1b4016cb3a46,27,17,4,12:53,1,40,0,0,Russell Wilson,Middle,0,0,0,4,2.20,Curl,Tyler Lockett,6,6,0,NaN,1,0,NaN,"R.Wilson pass complete to SEA 46. Catch made by T.Lockett at SEA 46. Gain of 6 yards. T.Lockett FUMBLES, forced by B.Breeland. Fumble RECOVERED by MIN-D.Hunter at SEA 44. Tackled by at SEA 44. The Replay Official reviewed the fumble and the play was overturned. R.Wilson pass short right complete to SEA 46. Catch made by T.Lockett at SEA 46. Gain of 6 yards. Tackled by B.Breeland at SEA 46."
64,6a04d980-1f20-11ec-a9c1-1b4016cb3a46,30,17,4,00:19,2,8,0,0,Russell Wilson,Middle,0,0,0,3,2.30,Out,Gerald Everett,9,17,8,1,1,1,NaN,R.Wilson pass short right complete to SEA 17. Catch made by G.Everett at SEA 17. Gain of 17 yards. Pushed out of bounds by B.Breeland at SEA 25.


In [114]:
sportradar_wr['hurry_up_attempt'] = False
sportradar_wr.loc[ (sportradar_wr['huddle'] == 'No Huddle'), 'hurry_up_attempt'] = True

In [115]:
sportradar_wr['hurry_up_catch'] = False
sportradar_wr.loc[ (sportradar_wr['huddle'] == 'No Huddle') & (sportradar_wr['reception'] == 1), 'hurry_up_catch'] = True

### Deep sideline catch
- 20 or more air yards
- Near one of the sidelines
- Gauges receiver's ability to be a vertical threat, maintain spatial awareness, and maintain body control when attempting the catch and staying inbounds

In [116]:
sportradar_wr.loc[(
                    (sportradar_wr['play_direction'] == 'Right Sideline') | (sportradar_wr['play_direction'] == 'Left Sideline')
                  )
                  & (sportradar_wr['att_yards'] >= 20)
                  & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                        'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 'att_yards', 'yards', 'yards_after_catch', 'yards_after_contact', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,yards_after_contact,on_target_throw,firstdown,touchdown,description
15,128b2eb0-1f0d-11ec-89e9-8992a3cbf855,7,10,2,14:09,1,35,0,0,Russell Wilson,Middle,0,0,0,4,2.58,Comeback,DK Metcalf,20,21,1,NaN,1,1,NaN,R.Wilson pass deep left complete to MIN 45. Catch made by D.Metcalf at MIN 45. Gain of 21 yards. Pushed out of bounds by X.Woods at MIN 44.


In [117]:
sportradar_wr['deep_sideline_attempt'] = False
sportradar_wr.loc[(
                    (sportradar_wr['play_direction'] == 'Right Sideline') | (sportradar_wr['play_direction'] == 'Left Sideline')
                  )
                  & (sportradar_wr['att_yards'] >= 20)
                  , 'deep_sideline_attempt'] = True

In [118]:
sportradar_wr['deep_sideline_catch'] = False
sportradar_wr.loc[(
                    (sportradar_wr['play_direction'] == 'Right Sideline') | (sportradar_wr['play_direction'] == 'Left Sideline')
                  )
                  & (sportradar_wr['att_yards'] >= 20)
                  & (sportradar_wr['reception'] == 1), 'deep_sideline_catch'] = True

### DPI drawn
- Receivers who frequently draw defensive pass interference calls worry defenders who don't want to surrender a big yardage gain

In [119]:
sportradar_wr.loc[sportradar_wr['defender_name'].notnull()
                 ][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                        'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 'att_yards', 'yards', 'yards_after_catch', 'yards_after_contact', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,yards_after_contact,on_target_throw,firstdown,touchdown,description


In [120]:
sportradar_wr['dpi_drawn'] = False
sportradar_wr.loc[sportradar_wr['defender_name'].notnull(), 'dpi_drawn'] = True

### Possession-saver catch
- A "possession receiver" is usually a tall, strong receiver who lacks speed, but can reliably make contested catches for short to intermediate yardage at the line to gain
- By catching passes intended for them at the line to gain, they sustain drives

In [121]:
sportradar_wr.loc[(sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd']) & (sportradar_wr['reception'] == 1)][['play_id', 'home_points', 'away_points', 'period', 'start_play_clock_string', 'start_play_down', 'start_play_yfd', 'start_play_yardline', 'inside_20', 'goaltogo', 'qb_name', 'pocket_location', 
                                                        'hurry', 'blitz', 'knockdown', 'players_rushed', 'pocket_time_string', 'pass_route', 'player_name', 'att_yards', 'yards', 'yards_after_catch', 'yards_after_contact', 'on_target_throw', 'firstdown', 'touchdown', 'description']]

,play_id,home_points,away_points,period,start_play_clock_string,start_play_down,start_play_yfd,start_play_yardline,inside_20,goaltogo,qb_name,pocket_location,hurry,blitz,knockdown,players_rushed,pocket_time_string,pass_route,player_name,att_yards,yards,yards_after_catch,yards_after_contact,on_target_throw,firstdown,touchdown,description
0,f72f3120-1f07-11ec-a67f-c75e589fa3a4,0,0,1,15:00,1,10,25,0,0,Russell Wilson,Boot Right,1,0,0,4,3.13,Cross,DK Metcalf,11,17,6,NaN,1,1,NaN,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.
2,8e3944c0-1f08-11ec-a67f-c75e589fa3a4,0,0,1,11:24,3,3,38,0,0,Russell Wilson,Middle,0,0,0,4,3.21,Post,DK Metcalf,11,28,17,2,1,1,NaN,R.Wilson pass short left complete to MIN 27. Catch made by D.Metcalf at MIN 27. Gain of 28 yards. Tackled by H.Smith at MIN 10.
5,9e800700-1f09-11ec-a67f-c75e589fa3a4,0,7,1,9:11,1,10,43,0,0,Kirk Cousins,Boot Right,0,1,0,5,3.25,Cross,Tyler Conklin,11,17,6,1,1,1,NaN,K.Cousins pass short left complete to SEA 32. Catch made by T.Conklin at SEA 32. Gain of 17 yards. Tackled by T.Flowers at SEA 26.
7,dabe15e0-1f09-11ec-b36d-a3b2acec3de6,0,7,1,7:54,2,2,18,1,0,Kirk Cousins,Middle,0,1,0,5,1.78,Out,Justin Jefferson,5,10,5,3,1,1,NaN,K.Cousins pass short left complete to SEA 13. Catch made by J.Jefferson at SEA 13. Gain of 10 yards. Tackled by T.Flowers at SEA 8.
9,ca06a590-1f0a-11ec-b36d-a3b2acec3de6,7,7,1,5:14,3,7,28,0,0,Russell Wilson,Middle,0,1,0,6,1.59,Slant,Freddie Swain,8,10,2,NaN,1,1,NaN,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.
10,e3c698a0-1f0a-11ec-b36d-a3b2acec3de6,7,7,1,4:38,1,10,38,0,0,Russell Wilson,Middle,0,1,1,5,2.25,Post,DK Metcalf,12,12,0,NaN,1,1,NaN,R.Wilson pass short middle complete to SEA 50. Catch made by D.Metcalf at SEA 50. Gain of 12 yards. Tackled by B.Breeland at MIN 50.
15,128b2eb0-1f0d-11ec-89e9-8992a3cbf855,7,10,2,14:09,1,10,35,0,0,Russell Wilson,Middle,0,0,0,4,2.58,Comeback,DK Metcalf,20,21,1,NaN,1,1,NaN,R.Wilson pass deep left complete to MIN 45. Catch made by D.Metcalf at MIN 45. Gain of 21 yards. Pushed out of bounds by X.Woods at MIN 44.
18,b25ceb80-1f0e-11ec-89e9-8992a3cbf855,7,17,2,7:25,1,10,38,0,0,Kirk Cousins,Middle,0,0,0,4,2.56,Post,Justin Jefferson,13,26,13,3,1,1,NaN,K.Cousins pass short middle complete to SEA 25. Catch made by J.Jefferson at SEA 25. Gain of 26 yards. Tackled by D.Reed at SEA 12.
21,19c341c0-1f0f-11ec-89e9-8992a3cbf855,13,17,2,5:32,3,13,15,1,0,Kirk Cousins,Middle,0,1,0,5,2.36,Post,Adam Thielen,15,15,0,NaN,1,1,1,"K.Cousins pass complete to SEA End Zone. Catch made by A.Thielen at SEA End Zone. Gain of 15 yards. A.Thielen for 15 yards, TOUCHDOWN."
23,0c9c7240-1f10-11ec-89e9-8992a3cbf855,14,17,2,4:41,2,3,28,0,0,Russell Wilson,Middle,0,1,0,5,2.46,Cross,Will Dissly,9,39,30,18,1,1,NaN,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.


In [122]:
sportradar_wr['possession_saver_attempt'] = False
sportradar_wr.loc[(sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd']), 'possession_saver_attempt'] = True

sportradar_wr['possession_saver_catch'] = False
sportradar_wr.loc[(sportradar_wr['att_yards'] >= sportradar_wr['start_play_yfd']) & (sportradar_wr['reception'] == 1), 'possession_saver_catch'] = True

# ADDITIONAL EXPLORATION
- Some more exploration of the data

In [123]:
sportradar_wr['play_id'].nunique(), sportradar_wr.shape
# Original shape was 67 x 124. Corrected it by adding the Out and Curl route attempt and catch columns. Now 67 x 128.

(67, (67, 128))

- Checking to see if all touchdown plays have "end zone" in the description.
- More than 2 TDs were scored in this game, so that appears not to be the case.

In [124]:
sportradar_wr[sportradar_wr['description'].str.contains('end zone', case = False, na = False)][['period', 'start_play_clock_string', 'start_play_yfd', 'start_play_yardline', 'start_play_field_side', 'player_name', 'att_yards', 'description']]

,period,start_play_clock_string,start_play_yfd,start_play_yardline,start_play_field_side,player_name,att_yards,description
21,2,5:32,13,15,SEA,Adam Thielen,15,"K.Cousins pass complete to SEA End Zone. Catch made by A.Thielen at SEA End Zone. Gain of 15 yards. A.Thielen for 15 yards, TOUCHDOWN."
32,2,00:20,3,3,SEA,Justin Jefferson,3,"K.Cousins pass complete to SEA End Zone. Catch made by J.Jefferson at SEA End Zone. Gain of 3 yards. J.Jefferson for 3 yards, TOUCHDOWN."


- Checking to see how the yardline adjustment looks.
- It appears to be working properly.
  - In row 9, the play ends with Seahawks, who are on offense, endin with the ball on their own 38
    - 38 is 12 yards away from midfield. Past midfield, Seattle has another 50 yards of opponent territory to cover before reaching the end zone.
    - The effective yardline is 12 + 50 = 62.

In [125]:
sportradar_wr[(sportradar_wr['team_alias'] == 'SEA') & (sportradar_wr['start_play_field_side'] == 'SEA')][['period', 'start_play_clock_string', 'start_play_yfd', 'start_play_yardline', 'effective_start_play_yardline', 'yards', 'end_play_yardline', 'effective_end_play_yardline', 
                                                                                                           'start_play_field_side', 'player_name', 'att_yards', 'description']]

,period,start_play_clock_string,start_play_yfd,start_play_yardline,effective_start_play_yardline,yards,end_play_yardline,effective_end_play_yardline,start_play_field_side,player_name,att_yards,description
0,1,15:00,10,25,75,17,42,58,SEA,DK Metcalf,11,R.Wilson pass short right complete to SEA 36. Catch made by D.Metcalf at SEA 36. Gain of 17 yards. Tackled by H.Smith at SEA 42.
9,1,5:14,7,28,72,10,38,62,SEA,Freddie Swain,8,R.Wilson pass short right complete to SEA 36. Catch made by F.Swain at SEA 36. Gain of 10 yards. Tackled by B.Breeland at SEA 38.
10,1,4:38,10,38,62,12,50,50,SEA,DK Metcalf,12,R.Wilson pass short middle complete to SEA 50. Catch made by D.Metcalf at SEA 50. Gain of 12 yards. Tackled by B.Breeland at MIN 50.
15,2,14:09,10,35,65,21,44,44,SEA,DK Metcalf,20,R.Wilson pass deep left complete to MIN 45. Catch made by D.Metcalf at MIN 45. Gain of 21 yards. Pushed out of bounds by X.Woods at MIN 44.
22,2,5:23,10,21,79,7,28,72,SEA,Penny Hart,-4,R.Wilson pass short middle complete to SEA 17. Catch made by P.Hart at SEA 17. Gain of 7 yards. Tackled by H.Smith at SEA 28.
23,2,4:41,3,28,72,39,33,33,SEA,Will Dissly,9,R.Wilson pass short left complete to SEA 37. Catch made by W.Dissly at SEA 37. Gain of 39 yards. Pushed out of bounds by E.Kendricks at MIN 33.
33,2,00:16,10,25,75,12,37,63,SEA,Travis Homer,5,"R.Wilson pass short middle complete to SEA 30. Catch made by T.Homer at SEA 30. Gain of 12 yards. Tackled by N.Vigil, E.Kendricks at SEA 37."
34,2,00:09,10,37,63,11,48,52,SEA,Gerald Everett,0,R.Wilson pass complete to SEA 37. Catch made by G.Everett at SEA 37. Gain of 11 yards. Pushed out of bounds by M.Alexander at SEA 48.
35,2,00:01,10,48,52,31,21,21,SEA,Travis Homer,-1,R.Wilson pass short left complete to SEA 47. Catch made by T.Homer at SEA 47. Gain of 31 yards. Tackled by C.Bynum at MIN 21.
43,3,4:38,19,27,73,5,32,68,SEA,Travis Homer,-6,R.Wilson pass short left complete to SEA 21. Catch made by T.Homer at SEA 21. Gain of 5 yards. Tackled by D.Hunter at SEA 32.


In [126]:
sportradar_wr['play_direction'].unique()

array(['Right', 'Left', 'Right Sideline', 'Left Sideline', 'Middle', nan],
      dtype=object)

In [127]:
sportradar_wr['qb_at_snap'].unique()

array(['Under Center', 'Shotgun', nan], dtype=object)

In [128]:
sportradar_wr['pass_route'].unique()

array(['Cross', 'Curl', 'Post', 'Out', 'Underneath Screen', 'Flat',
       'Slant', 'WR Screen', 'Comeback', 'Go', 'In', nan], dtype=object)

In [129]:
sportradar_wr.columns

Index(['game_id', 'season_year', 'season_type', 'week', 'home_alias',
       'home_sr_id', 'away_alias', 'away_sr_id', 'temp', 'humidity',
       'wind_speed', 'venue_surface', 'venue_roof_type', 'period', 'event_id',
       'team_sequence', 'play_id', 'home_points', 'away_points', 'description',
       'start_play_clock', 'start_play_clock_string', 'start_play_down',
       'start_play_yfd', 'start_play_yardline',
       'effective_start_play_yardline', 'start_play_field_side', 'inside_20',
       'goaltogo', 'end_play_clock', 'end_play_clock_string', 'end_play_down',
       'end_play_yfd', 'end_play_yardline', 'effective_end_play_yardline',
       'end_play_field_side', 'firstdown', 'scoring_play', 'players_rushed',
       'men_in_box', 'huddle', 'hash_mark', 'qb_at_snap', 'left_tightends',
       'right_tightends', 'qb_name', 'pocket_location', 'play_direction',
       'screen_pass', 'play_action', 'run_pass_option', 'pass_route',
       'fake_punt', 'fake_field_goal', 'defender_nam

# CONCLUSION
- We can now apply this series of cleaning and feature engineering steps to every game in the 2022 season.